## AIHub Search Strange Source in Json

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [1]:
import re
import os
import kss
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from pprint import pprint

In [2]:
pwd

'D:\\AIHUB'

### Function

In [3]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [4]:
def json_file_name_list(path_list):
    
    file_name  = [glob(i, recursive = True) for i in path_list][0]
    file_name = sorted_list(file_name)
    
    return file_name

In [105]:
def train_valid_json_file_name_list(path_list):

  train_file_name, valid_file_name = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_name = sorted_list(train_file_name)
  valid_file_name = sorted_list(valid_file_name)
    
  return train_file_name, valid_file_name

In [8]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [9]:
def formal_preprocessing_text_kss1(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)
  
    return preprocessing_sentence_list

In [86]:
def formal_preprocessing_text_kss3(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

In [11]:
def webdata_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
            
                sentence = sentence.replace(". . .", ".")
                sentence = sentence.replace(". .", ".")
                sentence = sentence.replace("..", ".")
                
            # 구두점 ". . .", ". .", ".." 을 온점"."으로 교체

                preprocessing_sentence_list.append(sentence)
  
    return preprocessing_sentence_list


In [12]:
def legislation_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거   
    
    sources = re.split(r"[0-9]+[.]|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮", source)
    # 숫자.(1. 2. ...), 원숫자(①, ②, ③, ...)를 기준으로 문장 분리

    for source in sources:
        
        for sentence in kss.split_sentences(source, use_heuristic=False,
                                            num_workers=32):
        # KSS(Korean Sentence Segmentation)로 문장 분리 
        # Formal articles (wiki, news, essays): recommend to False


            if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
                bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
                len(sentence.split()) > 5:
                # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
                # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
                # 다섯 어절 이하 문장 제외


                if ']' in sentence and '[' not in sentence:
                    sentence  = re.sub(r".*?]", "", sentence)    
                # 중괄호 앞에 있는 '성명/직함]' 형태 제거


                sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
                # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

                sentence = sentence.strip()
                # strip으로 앞뒤 공백 제거

                total_length = len(sentence.replace(" " , ""))
                hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
                hangeul_ratio = hangeul_length / total_length
                if hangeul_ratio >= 0.5:
                # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

In [13]:
def legal_preprocessing_text(source):
    preprocessing_sentence_list = []
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
   
    source = re.sub(r'제\d{1,}조의\d{1,}\([^)]+\)', "", source)
    source = re.sub(r'제\d{1,}조\([^)]+\)', "",source)
    sources = re.split(r"[0-9]+[.]|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮", source)
    # '제n조의n항(text)', '제n조(text)' 꼴 제거
    # 숫자.(1. 2. ...), 원숫자(①, ②, ③, ...)를 기준으로 문장 분리
    
    for source in sources:
    
        for sentence in kss.split_sentences(source, use_heuristic=False,
                                            num_workers=32):
        # KSS(Korean Sentence Segmentation)로 문장 분리 
        # Formal articles (wiki, news, essays): recommend to False

            if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮]", sentence[0]) is not None and \
                bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
                len(sentence.split()) > 5:
                # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
                # ①, ②, ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨, ⑩, ⑪, ⑫, ⑬, ⑭, ⑮ 제외) 제외
                # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
                # 다섯 어절 이하 문장 제외

                sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
                # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

                sentence = sentence.strip()
                # strip으로 앞뒤 공백 제거

                total_length = len(sentence.replace(" " , ""))
                hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
                hangeul_ratio = hangeul_length / total_length
                if hangeul_ratio >= 0.5:
                # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

In [14]:
def tableqa_preprocessing_text(source):
    
    preprocessing_sentence_list = []
    source_split = re.split('br|tr|td|table|tbody|rowspan', source)
    # 문자열 'br', 'tr', 'td', 'table', 'tbody'를 기준으로 문장 분리

    for source in source_split:
        sentence = re.sub('(br|tr|td|table|tbody|rowspan|<|>|/)', '', source)
        # 문자열 'br', 'tr', 'td', 'table', 'tbody' 특수기호 '<', '>', '/'를 제외
            
        sentence = sentence.strip()
        # strip으로 앞뒤 공백 제거
        
        sentence = re.sub(r"\[.*?\]|\{.*?\}", "", sentence)
        # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
        
       
        
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(sentence)
        
        
        try:

            if type(text_in_small_bracket) == str:

                text = text_in_small_bracket

                text_size = len(text)
                last_index = sentence.find(text) + len(text)
                if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                    sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    sentence = sentence.replace(small_bracket, text)    

            elif type(text_in_small_bracket) == list:

                for text in text_in_small_bracket:

                    text_size = len(text)
                    last_index = sentence.find(text) + len(text)
                    if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                        sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

                    if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                        small_bracket = "(" + text + ")"
                        sentence = sentence.replace(small_bracket, text)    

        except:
            pass

            # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
            # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
        

        if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', sentence[:2])) == True:
            sentence = source.replace(source[:2], "")

        sentence = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", sentence)
        # '가.', '나.', ... 형태의 문자열 제거 


        if len(sentence) > 1 and \
        re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
        bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and  \
        len(sentence.split()) > 5:
         # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
        # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
        # 다섯 어절 이하 문장 제외

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

### AIHUB 전문분야 말뭉치

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=110)

In [653]:
path_list = ['AIHUB_전문분야 말뭉치/Training/'+ '/**/*.json', 
             'AIHUB_전문분야 말뭉치/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [654]:
source_file = train_file_name[1]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_02.json


In [ ]:
num = 5000

for i in range(num-100, num):
    try : 
        if source_list[i][-1] == ".":
            preprocessing_sentences = formal_preprocessing_text_kss1(source_list[i])
            if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i] and \
            "；" not in source_list[i] and len(preprocessing_sentences) >= 2:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
    except:
        pass

In [655]:
num = 5000

for i in range(num-100, num):
    if '[해결수단]' in source_list[i]:
        print("Index:", i)
        print()
        print(source_list[i])
        print()

Index: 4930

[과제] 스커트 가드 내부에 간단하게 설치할 수 있고, 이용자의 넘어짐을 확실히 검지할 수 있는 승객 컨베이어의 넘어짐 검지장치를 얻는다. [해결수단] 스커트 가드의 빗판 부분을 감시할 수 있는 위치에 마련된 개구부와, 개구부의 배면 측에 마련 레이저 투과창과, 레이저 투과창의 배면 측의 스커트 가드내에 설치된 레이저 스캔센서 유니트를 구비하고, 레이저 스캔센서 유니트는 레이저 투과창에 대응하는 부분이 개방된 케이스와, 케이스 내에 마련되고, 레이저 광을 방사하는 레이서 스캔형 거리센서와, 레이저 스캔형 거리센서에 마련되고, 레이저 투과창을 향하여 레이저 광을 송신하는 송신부와, 레이저 스캔형 거리센서에 마련되고, 레이저 투과창을 통해 반사되어 오는 레이저 광을 수신하는 수신부와, 송신부와 수신부의 사이에 마련되고, 스커트 가드 내부의 반사에 의해 수신부에 도달하지 않도록 하는 구획판을 구비한다

Index: 4993

[과제] 반도체 장치나 마이크로 머신과 같은 미세 구조체의 패턴 붕괴를 억제할 수 있는 처리액 및 이것을 이용한 미세 구조체의 제조방법을 제공한다. [해결수단] 탄소수 6 ~ 18의 알킬기를 가지는 알킬포스폰산 또는 그 염과 물 및 수용성 용제를 함유하는 금속 미세 구조체의 패턴 붕괴 억제용 처리액으로 처리한다.



In [656]:
print(source_list[4993])

[과제] 반도체 장치나 마이크로 머신과 같은 미세 구조체의 패턴 붕괴를 억제할 수 있는 처리액 및 이것을 이용한 미세 구조체의 제조방법을 제공한다. [해결수단] 탄소수 6 ~ 18의 알킬기를 가지는 알킬포스폰산 또는 그 염과 물 및 수용성 용제를 함유하는 금속 미세 구조체의 패턴 붕괴 억제용 처리액으로 처리한다.


In [657]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[4993])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

반도체 장치나 마이크로 머신과 같은 미세 구조체의 패턴 붕괴를 억제할 수 있는 처리액 및 이것을 이용한 미세 구조체의 제조방법을 제공한다.

탄소수 6   18의 알킬기를 가지는 알킬포스폰산 또는 그 염과 물 및 수용성 용제를 함유하는 금속 미세 구조체의 패턴 붕괴 억제용 처리액으로 처리한다.



In [658]:
print(preprocessing_sentences)

['반도체 장치나 마이크로 머신과 같은 미세 구조체의 패턴 붕괴를 억제할 수 있는 처리액 및 이것을 이용한 미세 구조체의 제조방법을 제공한다.', '탄소수 6   18의 알킬기를 가지는 알킬포스폰산 또는 그 염과 물 및 수용성 용제를 함유하는 금속 미세 구조체의 패턴 붕괴 억제용 처리액으로 처리한다.']


In [61]:
source_file = train_file_name[18]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_z_1.json


In [95]:
num = 1130

for i in range(num-10, num):
    try : 
        if source_list[i][-1] == ".":
            preprocessing_sentences = formal_preprocessing_text_kss1(source_list[i])
            if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
    except:
        pass

[Before]
적어도 산화인듐 및 산화갈륨을 함유하는 소결체로서, 체적 14000μm3 이상의 공극이 0.03체적% 이하인 소결체.


[After]
적어도 산화인듐 및 산화갈륨을 함유하는 소결체로서, 체적 14000 m3 이상의 공극이 0.03체적  이하인 소결체.


[Before]
본 발명은 저반사 고경도 코팅막 및 이를 이용한 표시 장치에 관한 것으로, 본 발명의 저반사 고경도 코팅막은 고경도를 위해 폴리실록산과 실리카 혼성으로 네트워크를 구성하며 다수의 미세기공을 포함하는 다공성 실리카 나노층 구조를 포함하고; 상기 다공성 실리카 나노층은 실리카 및 에어 부피비 조절을 통해 1.3 이하의 저굴절율을 갖는다.


[After]
본 발명은 저반사 고경도 코팅막 및 이를 이용한 표시 장치에 관한 것으로, 본 발명의 저반사 고경도 코팅막은 고경도를 위해 폴리실록산과 실리카 혼성으로 네트워크를 구성하며 다수의 미세기공을 포함하는 다공성 실리카 나노층 구조를 포함하고  상기 다공성 실리카 나노층은 실리카 및 에어 부피비 조절을 통해 1.3 이하의 저굴절율을 갖는다.


[Before]
본 발명의 실시 예에 따른 후판 교정기용 영점조절패널 지지장치는 설정된 간격으로 상호 평행하게 배치된 제1 및 제2영점조절패널의 선단부를 상호 연결하는 제1지지바; 제1 및 제2영점조절패널의 후단부를 상호 연결하며 제1지지바와 평행하게 배치되는 제2지지바; 제1지지바와 제2지지바를 연결하는 연결부; 및 제1지지바와 제1 및 제2영점조절패널 선단부의 체결 또는 제2지지바와 제1 및 제2영점조절패널 후단부의 체결을 위한 복수의 체결부;를 포함한다.


[After]
본 발명의 실시 예에 따른 후판 교정기용 영점조절패널 지지장치는 설정된 간격으로 상호 평행하게 배치된 제1 및 제2영점조절패널의 선단부를 상호 연결하는 제1지지바  제1 및 제2영점조절패널의 후단부를 상호 연결하며 제1지지바와 평행하게 배치되는 제2지지바  제1지지바와 제2지지바를 연결하는 연결부  및 제1지지바와 제1 

In [71]:
print(source_list[28])

시클로헥사논 옥심 생성을 위한 암옥심화 반응기는, (a) 교반기를 구비한 반응기 용기; (b) 내부 여과 시스템; (c) 내부 액체 암모니아 증발 코일; (d) 내부 기체 암모니아 환형 분배기; (e) 외부 시클로헥사논 환형 분배기; (f) 내부 과산화수소 환형 분배기; (g) 내부 원통형 흡출관 (draft tube); 및 (h) 외부 냉각 재킷을 포함한다.


In [72]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[28])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

시클로헥사논 옥심 생성을 위한 암옥심화 반응기는, (a) 교반기를 구비한 반응기 용기  (b) 내부 여과 시스템  (c) 내부 액체 암모니아 증발 코일  (d) 내부 기체 암모니아 환형 분배기  (e) 외부 시클로헥사논 환형 분배기  (f) 내부 과산화수소 환형 분배기  (g) 내부 원통형 흡출관 (draft tube)  및 (h) 외부 냉각 재킷을 포함한다.



In [98]:
source_file = train_file_name[17]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\논문_1_1.json


In [ ]:
num = 1300

for i in range(num-10, num):
    try : 
        if source_list[i][-1] == ".":
            preprocessing_sentences = formal_preprocessing_text_kss1(source_list[i])
            if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
    except:
        pass

In [131]:
print(source_list[1298])

B형 인플루엔자 감염군에서 발열 기간이 유의하게 길었다(P =0.001).


In [132]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[1298])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

B형 인플루엔자 감염군에서 발열 기간이 유의하게 길었다.



In [133]:
print(preprocessing_sentences)

['B형 인플루엔자 감염군에서 발열 기간이 유의하게 길었다.']


In [29]:
source_file = train_file_name[30]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\법령_l_1302_1.json


In [168]:
num = 350

for i in range(num-5, num):
    try : 
        if source_list[i][-1] == ".":
            preprocessing_sentences = legal_preprocessing_text(source_list[i])
            if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
    except:
        pass

[Before]
제46조(보수 결정의 원칙) ① 공무원의 보수는 직무의 곤란성과 책임의 정도에 맞도록 계급별ㆍ직위별 또는 직무등급별로 정한다. 다만, 직무의 곤란성과 책임도가 매우 특수하거나 결원을 보충하는 것이 곤란한 직무에 종사하는 공무원과 제4조 제2항에 따라 같은 조 제1항의 계급 구분을 적용하지 아니하는 공무원의 보수는 따로 정할 수 있다.


[After]
공무원의 보수는 직무의 곤란성과 책임의 정도에 맞도록 계급별 직위별 또는 직무등급별로 정한다.


다만, 직무의 곤란성과 책임도가 매우 특수하거나 결원을 보충하는 것이 곤란한 직무에 종사하는 공무원과 제4조 제2항에 따라 같은 조 제1항의 계급 구분을 적용하지 아니하는 공무원의 보수는 따로 정할 수 있다.


[Before]
② 공무원의 보수는 일반의 표준 생계비, 물가 수준, 그 밖의 사정을 고려하여 정하되, 민간 부문의 임금 수준과 적절한 균형을 유지하도록 노력하여야 한다.


[After]
공무원의 보수는 일반의 표준 생계비, 물가 수준, 그 밖의 사정을 고려하여 정하되, 민간 부문의 임금 수준과 적절한 균형을 유지하도록 노력하여야 한다.




In [170]:
print(source_list[351])

제47조(보수에 관한 규정) ① 공무원의 보수에 관한 다음 각 호의 사항은 대통령령으로 정한다.


In [171]:
preprocessing_sentences = legal_preprocessing_text(source_list[351])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

공무원의 보수에 관한 다음 각 호의 사항은 대통령령으로 정한다.



In [174]:
print(preprocessing_sentences)

['공무원의 보수에 관한 다음 각 호의 사항은 대통령령으로 정한다.']


In [33]:
source_file = train_file_name[24]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\판례_case03.json


In [43]:
num = 120

for i in range(num-5, num):
    try : 
        if source_list[i][-1] == ".":
            preprocessing_sentences = legal_preprocessing_text(source_list[i])
            if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
    except:
        pass

[Before]
가.의사자격이 있는 사람으로서 군전공의수련기관에서 정하여진 과정을 마치고 자연계대학원에서 박사학위 과정을 수학 중인 사람이 35세까지 의무종사기간을 마칠 수 있는 경우에 한하여 전문연구요원으로 편입할 수 있도록 규정한 구 병역법(2013. 6. 4. 법률 제11849호로 개정되기 전의 것) 제36조 제5항에 대하여 헌법소원심판을 청구한 이후, 2013. 6. 4. 법률 제11849호로 위 조항이 개정되어 '의사 등의 자격이 있는 사람으로서 군전공의수련기관에서 정하여진 과정을 마치고 자연계대학원에서 박사학위 과정을 수학 중인 사람'의 경우에는 '37세'까지 의무종사기간을 마칠 수 있는 사람은 전문연구요원으로 편입될 수 있도록 변경되어 청구인도 전문연구요원으로 편입할 수 있게 되었으므로, 위 조항에 대한 헌법소원심판청구는 권리보호이익이 없다.


[After]
법률 제11849호로 위 조항이 개정되어  의사 등의 자격이 있는 사람으로서 군전공의수련기관에서 정하여진 과정을 마치고 자연계대학원에서 박사학위 과정을 수학 중인 사람 의 경우에는  37세 까지 의무종사기간을 마칠 수 있는 사람은 전문연구요원으로 편입될 수 있도록 변경되어 청구인도 전문연구요원으로 편입할 수 있게 되었으므로, 위 조항에 대한 헌법소원심판청구는 권리보호이익이 없다.


[Before]
나.이 사건 편입대상조항에서 의사자격이 있는 사람으로서 군전공의 수련과정을 마치고 박사학위 과정에 수학 중인 사람을 전문연구요원 편입대상으로 규정하고 있는 것은 이들의 임상경험과 첨단 과학기술을 접목시켜 의료기기 산업 등을 발전시키고 의과학자를 양성하는 데 그 목적이 있다.


[After]
이 사건 편입대상조항에서 의사자격이 있는 사람으로서 군전공의 수련과정을 마치고 박사학위 과정에 수학 중인 사람을 전문연구요원 편입대상으로 규정하고 있는 것은 이들의 임상경험과 첨단 과학기술을 접목시켜 의료기기 산업 등을 발전시키고 의과학자를 양성하는 데 그 목적이 있다.




In [45]:
print(source_list[119])

나.이 사건 편입대상조항에서 의사자격이 있는 사람으로서 군전공의 수련과정을 마치고 박사학위 과정에 수학 중인 사람을 전문연구요원 편입대상으로 규정하고 있는 것은 이들의 임상경험과 첨단 과학기술을 접목시켜 의료기기 산업 등을 발전시키고 의과학자를 양성하는 데 그 목적이 있다.


In [46]:
preprocessing_sentences = legal_preprocessing_text(source_list[119])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

이 사건 편입대상조항에서 의사자격이 있는 사람으로서 군전공의 수련과정을 마치고 박사학위 과정에 수학 중인 사람을 전문연구요원 편입대상으로 규정하고 있는 것은 이들의 임상경험과 첨단 과학기술을 접목시켜 의료기기 산업 등을 발전시키고 의과학자를 양성하는 데 그 목적이 있다.



In [47]:
print(preprocessing_sentences)

['이 사건 편입대상조항에서 의사자격이 있는 사람으로서 군전공의 수련과정을 마치고 박사학위 과정에 수학 중인 사람을 전문연구요원 편입대상으로 규정하고 있는 것은 이들의 임상경험과 첨단 과학기술을 접목시켜 의료기기 산업 등을 발전시키고 의과학자를 양성하는 데 그 목적이 있다.']


### AIHUB 대규모 웹데이터 기반 한국어 말뭉치 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624)

In [18]:
path_list = ['AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/IT_과학/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/건강/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/경제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/교육/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/국제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/라이프스타일/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/문화/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/사건사고/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/사회일반/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/산업/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/스포츠/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/여성복지/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/여행레저/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/연예/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/정치/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/지역/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/취미/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/IT_과학/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/건강/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/경제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/교육/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/국제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/라이프스타일/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/문화/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/사건사고/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/사회일반/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/산업/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/스포츠/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/여성복지/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/여행레저/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/연예/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/정치/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/지역/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/취미/' + '**/*.json',]

train_file_name_01 = glob(path_list[0], recursive = True)
train_file_name_02 = glob(path_list[1], recursive = True)
train_file_name_03 = glob(path_list[2], recursive = True)
train_file_name_04 = glob(path_list[3], recursive = True)
train_file_name_05 = glob(path_list[4], recursive = True)
train_file_name_06 = glob(path_list[5], recursive = True)
train_file_name_07 = glob(path_list[6], recursive = True)
train_file_name_08 = glob(path_list[7], recursive = True)
train_file_name_09 = glob(path_list[8], recursive = True)
train_file_name_10 = glob(path_list[9], recursive = True)
train_file_name_11 = glob(path_list[10], recursive = True)
train_file_name_12 = glob(path_list[11], recursive = True)
train_file_name_13 = glob(path_list[12], recursive = True)
train_file_name_14 = glob(path_list[13], recursive = True)
train_file_name_15 = glob(path_list[14], recursive = True)
train_file_name_16 = glob(path_list[15], recursive = True)
train_file_name_17 = glob(path_list[16], recursive = True)

valid_file_name_01 = glob(path_list[17], recursive = True)
valid_file_name_02 = glob(path_list[18], recursive = True)
valid_file_name_03 = glob(path_list[19], recursive = True)
valid_file_name_04 = glob(path_list[20], recursive = True)
valid_file_name_05 = glob(path_list[21], recursive = True)
valid_file_name_06 = glob(path_list[22], recursive = True)
valid_file_name_07 = glob(path_list[23], recursive = True)
valid_file_name_08 = glob(path_list[24], recursive = True)
valid_file_name_09 = glob(path_list[25], recursive = True)
valid_file_name_10 = glob(path_list[26], recursive = True)
valid_file_name_11 = glob(path_list[27], recursive = True)
valid_file_name_12 = glob(path_list[28], recursive = True)
valid_file_name_13 = glob(path_list[29], recursive = True)
valid_file_name_14 = glob(path_list[30], recursive = True)
valid_file_name_15 = glob(path_list[31], recursive = True)
valid_file_name_16 = glob(path_list[32], recursive = True)
valid_file_name_17 = glob(path_list[33], recursive = True)

In [19]:
source_file = train_file_name_03[11]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_list = list(pd.DataFrame(one_json_sample['SJML']['text'])['content'])

AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/경제\BWEC217000004648.json


In [ ]:
num = 500

for i in range(num-500, num):
    try : 
        if source_list[i][-1] == "." and source_list[i][0] != ".":
            preprocessing_sentences = webdata_preprocessing_text(source_list[i])
            if type(preprocessing_sentences) == list and len(preprocessing_sentences) <= 6 and preprocessing_sentences[0] != source_list[i]:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
    except:
        pass

In [21]:
print(source_list[4])

하나마이크론이 1분기 양호한 실적을 냈다는 소식에 강세다.. . 28일 오전 10시41분 현재 코스닥시장에서 하나마이크론은 전 거래일보다 450원(4.41%) 상승한 1만650원에 거래되고 있다.. . 하나마이크론은 이날 1분기 잠정영업실적 집계결과 매출액은 709억원으로 전년 동기 대비 10.1% 늘었다고 공시했다. 영업이익은 61억원으로 전년 동기보다 95.8% 증가했으며 당기순이익은 154.9% 늘어난 43억원을 달성했다.. . 실적 성장의 주 배경은 ▲lpddr4 등 최신 모바일 d램을 포함한 고부가치 메모리 패키지 수주확대 ▲반도체 부품과 공정용 가스를 생산하는 관계사 하나머티리얼즈의 수익성 개선 등으로 요약된다. . . 하나마이크론 관계자는 “고부가 패키징 수주 물량 확대 지속과 관계사들의 실적 회복으로 실적 성장이 지속 될 것”이라고 밝혔다. .


In [24]:
preprocessing_sentences = webdata_preprocessing_text(source_list[4])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

하나마이크론이 1분기 양호한 실적을 냈다는 소식에 강세다.

28일 오전 10시41분 현재 코스닥시장에서 하나마이크론은 전 거래일보다 450원(4.41 ) 상승한 1만650원에 거래되고 있다.

하나마이크론은 이날 1분기 잠정영업실적 집계결과 매출액은 709억원으로 전년 동기 대비 10.1  늘었다고 공시했다.

영업이익은 61억원으로 전년 동기보다 95.8  증가했으며 당기순이익은 154.9  늘어난 43억원을 달성했다.

실적 성장의 주 배경은  lpddr4 등 최신 모바일 d램을 포함한 고부가치 메모리 패키지 수주확대  반도체 부품과 공정용 가스를 생산하는 관계사 하나머티리얼즈의 수익성 개선 등으로 요약된다.

하나마이크론 관계자는  고부가 패키징 수주 물량 확대 지속과 관계사들의 실적 회복으로 실적 성장이 지속 될 것 이라고 밝혔다.



In [25]:
print(preprocessing_sentences)

['하나마이크론이 1분기 양호한 실적을 냈다는 소식에 강세다.', '28일 오전 10시41분 현재 코스닥시장에서 하나마이크론은 전 거래일보다 450원(4.41 ) 상승한 1만650원에 거래되고 있다.', '하나마이크론은 이날 1분기 잠정영업실적 집계결과 매출액은 709억원으로 전년 동기 대비 10.1  늘었다고 공시했다.', '영업이익은 61억원으로 전년 동기보다 95.8  증가했으며 당기순이익은 154.9  늘어난 43억원을 달성했다.', '실적 성장의 주 배경은  lpddr4 등 최신 모바일 d램을 포함한 고부가치 메모리 패키지 수주확대  반도체 부품과 공정용 가스를 생산하는 관계사 하나머티리얼즈의 수익성 개선 등으로 요약된다.', '하나마이크론 관계자는  고부가 패키징 수주 물량 확대 지속과 관계사들의 실적 회복으로 실적 성장이 지속 될 것 이라고 밝혔다.']


### AIHUB 산업정보 연계 주요국 특허 영-한 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=563)

In [302]:
path_list = ['AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training/'+ '/**/*.json', 
             'AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [319]:
source_file = train_file_name[438]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
try: 
    source_list = list(pd.DataFrame(one_json_sample['labeled_data'])['astrt_cont_kor'])

except:
    pass   

AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_3\생물 살균ㆍ살충제 및 식물보호제 제조업.json


In [ ]:
num = len(source_list)
english_form = re.compile('[a-z]+')

        
for i in range(0, num):
    try : 
        english = english_form.findall(source_list[i])
        if len(english) >= 10:
            preprocessing_sentences = formal_preprocessing_text_kss1(source_list[i])
            if type(preprocessing_sentences) == list and len(preprocessing_sentences) > 1 and preprocessing_sentences[0] != source_list[i]:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
    except:
        pass

In [321]:
num = -1
for source in source_list:
    num += 1
    if "본 발명은 선택된 박테리아성 유해 유기체를 유용한 식물" in source:
        print(num)

113


In [322]:
print(source_list[113])

본 발명은 선택된 박테리아성 유해 유기체를 유용한 식물에서 방제하기 위한 숙주 방어 유도물질의 용도에 관한 것으로, 여기에서 박테리아성 유해 유기체는 아시도보락스 아베나에(Acidovoraxavenae), 버크홀데리아(Burkholderia) spec., 버크홀데리아 글루마에(Burkholderiaglumae), 칸디다투스 리베리박터(Candidatus Liberibacter) spec., 코리네박테리움(Corynebacterium), 에르위니아(Erwinia) spec., 슈도모나스 시링가에(Pseudomonassyringae), 슈도모나스 시링가에 pv. 악티니디아에(Pseudomonassyringaepv. actinidae), 슈도모나스 시링가에 pv. 글라이시네아(Pseudomonassyringaepv. glycinea), 슈도모나스 시링가에 pv. 토마토(Pseudomonassyringaepv. tomato), 슈도모나스 시링가에 pv. 라크리만스(Pseudomonassyringae pv. lachrymans), 스트렙토마이세스(Streptomyces) spp., 잔토모나스(Xanthomonas) spp., 잔토모나스 악조노포디스(Xanthomonas axonopodis), 잔토모나스 악조노포디스 pv. 시트리(Xanthomonasaxonopodis pv. citri), 잔토모나스 악조노포디스 pv. 글라이시네스(Xanthomonasaxonopodis pv. glycines), 잔토모나스 캄페스트리스(Xanthomonascampestris), 잔토모나스 캄페스트리스 pv. 무사시애룸(Xanthomonascampestris pv. musacearum), 잔토모나스 캄페스트리스 pv. 프루니(Xanthomonascampestrispv. pruni), 잔토모나스 프라가리아에(Xanthomonasfragariae) 및 잔토모나스 트랜스루센스(Xanthomonas transluscens)로 구성되는 군에서 선택된다. 본 발명의 바람직한 측면에서 숙주 방어 유도

In [323]:
preprocessing_sentences = legal_preprocessing_text(source_list[113])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

본 발명의 바람직한 측면에서 숙주 방어 유도물질은 이소티아닐 (isotianil)이다.

본 발명은 또한 숙주 방어 유도물질로 처리하여 선택된 박테리아성 유해 유기체를 유용한 식물에서 방제하는 방법에 관한 것이다.



In [324]:
print(preprocessing_sentences)

['본 발명의 바람직한 측면에서 숙주 방어 유도물질은 이소티아닐 (isotianil)이다.', '본 발명은 또한 숙주 방어 유도물질로 처리하여 선택된 박테리아성 유해 유기체를 유용한 식물에서 방제하는 방법에 관한 것이다.']


### AIHUB 논문자료 요약

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=90)

In [26]:
path_list = ['AIHUB_논문자료 요약/Training/'+ '/**/*.json', 
             'AIHUB_논문자료 요약/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [55]:
source_file = train_file_name[5]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
    
if '논문/논문요약' or '특허섹션' in source_file:
    try:
        summary_entire_source_dict = dict(source_df['summary_entire'].explode())
        summary_entire_source_json = pd.json_normalize(summary_entire_source_dict)
        summary_entire_source_list = list(summary_entire_source_json.filter(regex='orginal_text').values[0])

        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        summary_section_source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

        source_list = summary_entire_source_list + summary_section_source_list
   
    except:
        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])
        
else:
    summary_section_source_dict = dict(source_df['summary_section'].explode())
    summary_section_source_json = pd.json_normalize(summary_section_source_dict)
    source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

AIHUB_논문자료 요약/Training\training_논문\논문요약_0206_2.json


#### 전체요약

In [ ]:
num = 3000

for i in range(num-1000, num):
    try : 
        preprocessing_sentences = formal_preprocessing_text_kss1(source_list[i])
        if type(preprocessing_sentences) == list and len(preprocessing_sentences) == 3 and preprocessing_sentences[0] != source_list[i]:
            print("[Before]")
            print(source_list[i])
            print("\n")

            print("[After]")
            for preprocessing_sentence in preprocessing_sentences:          
                    print(preprocessing_sentence, end="\n")
                    print("\n")
    except:
        pass

In [58]:
num = -1
for source in source_list:
    num += 1
    if "南明泉和尙頌證道歌" in source:
        print(num)

1386
7645


In [61]:
print(source_list[1386])

이 연구는 「南明泉和尙頌證道歌」를 찍었던 고려활자와 관련된 계속 연구로 이미 알려져 있는 分司大藏都監에서 간행된 「東國李相國集」 全集의 版本도 그 底本은 證道歌字로 찍었다는 사실을 밝히는 내용이다. 더 나아가 「東國李相國集」 全集을 찍는데 사용된 활자의 글씨와 證道歌字가 같다면 기존에 추정되었던 여러 가지 사실중 「古今詳定禮文」은 새로운 활자를 주조하여 찍은 것이 아니며 소위 「證道歌」, 「詳定禮文」, 「李相國集」이 10여년 이내에 같은 활자로 찍었고 활자 역시 開城에서 주조되어 江華島 遷都 때 함께 가지고 가서 책의 출판에 사용하다가 1270년경 다시 還都할 때 돌아온 것으로 추정되는 배경과 근거를 제시하였다. 한편 많은 양의 실물활자들이 나타남에 따라 그동안 국립중앙박물관 소장의 ‘복’자나 북한의 ‘전’자 등 고려활자에 대하여 중국학자들이 주장한 몇 가지 측면의 부정적 주장을 불식시키는 자료로 이 활자들이 활용될 수 있을 것이다.


In [64]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[1386])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

이 연구는  南明泉和尙頌證道歌 를 찍었던 고려활자와 관련된 계속 연구로 이미 알려져 있는 分司大藏都監에서 간행된  東國李相國集  全集의 版本도 그 底本은 證道歌字로 찍었다는 사실을 밝히는 내용이다.

더 나아가  東國李相國集  全集을 찍는데 사용된 활자의 글씨와 證道歌字가 같다면 기존에 추정되었던 여러 가지 사실중  古今詳定禮文 은 새로운 활자를 주조하여 찍은 것이 아니며 소위  證道歌 ,  詳定禮文 ,  李相國集 이 10여년 이내에 같은 활자로 찍었고 활자 역시 開城에서 주조되어 江華島 遷都 때 함께 가지고 가서 책의 출판에 사용하다가 1270년경 다시 還都할 때 돌아온 것으로 추정되는 배경과 근거를 제시하였다.

한편 많은 양의 실물활자들이 나타남에 따라 그동안 국립중앙박물관 소장의  복 자나 북한의  전 자 등 고려활자에 대하여 중국학자들이 주장한 몇 가지 측면의 부정적 주장을 불식시키는 자료로 이 활자들이 활용될 수 있을 것이다.



In [65]:
print(preprocessing_sentences)

['이 연구는  南明泉和尙頌證道歌 를 찍었던 고려활자와 관련된 계속 연구로 이미 알려져 있는 分司大藏都監에서 간행된  東國李相國集  全集의 版本도 그 底本은 證道歌字로 찍었다는 사실을 밝히는 내용이다.', '더 나아가  東國李相國集  全集을 찍는데 사용된 활자의 글씨와 證道歌字가 같다면 기존에 추정되었던 여러 가지 사실중  古今詳定禮文 은 새로운 활자를 주조하여 찍은 것이 아니며 소위  證道歌 ,  詳定禮文 ,  李相國集 이 10여년 이내에 같은 활자로 찍었고 활자 역시 開城에서 주조되어 江華島 遷都 때 함께 가지고 가서 책의 출판에 사용하다가 1270년경 다시 還都할 때 돌아온 것으로 추정되는 배경과 근거를 제시하였다.', '한편 많은 양의 실물활자들이 나타남에 따라 그동안 국립중앙박물관 소장의  복 자나 북한의  전 자 등 고려활자에 대하여 중국학자들이 주장한 몇 가지 측면의 부정적 주장을 불식시키는 자료로 이 활자들이 활용될 수 있을 것이다.']


#### 섹션 요약

In [66]:
num = -1
for source in source_list:
    num += 1
    if "이 연구는 필자가 2010년 9월에 발표한" in source:
        print(num)

7645


In [67]:
print(source_list[7645])

이 연구는 필자가 2010년 9월에 발표한 『南明泉和尙頌證道歌』를 찍었던 고려활자와 관련된 계속 연구로 이미 알려져 있는 分司大藏都監에서 간행된 『東國李相國集』 全集의 版本도 그 底本은 證道歌字로 찍었다는 사실을 밝히는 내용이다. 그 동안 새로 발견된 고려 금속활자에 대한 다방면에서의 기초적 검토는 이미 발표되었으며 2010년 ‘書誌學報’에 수록된 필자의 ‘證道歌字의 발견과 『南明泉和尙頌證道歌』의 연구’라는 논문에서 활자의 형태적 특징, 서체 및 탄소연대측정에 대한 분석 결과를 이미 언급한 바 있다. 또한 필자는 2004년부터 이 활자로 찍은 印本에 대하여 조사하면서 그동안 활자본의 飜刻 여부에 대하여 계속적인 의문을 가지고 있었던 『東國李相國集』을 다시 검토하고 그 과정에서 수년 전에 번각본의 가능성을 제시하였던 윤병태 교수님의 선행연구를 살펴볼 수 있었다. 지금까지 각 연구영역에서 다루어졌던 『東國李相國集』은 주로 역사학, 문학, 불교학 등에서 저자인 李奎報에 대한 傳記的 내용과 함께 기본자료로서 많이 이용되고 연구되어 왔다. 또 書誌的인 측면에서도 刊行과 版本에 대하여 부분적으로 다루어진 연구도 있었다. 그러나 초기의 판본들 중에는 補刻板의 混入이 많고 그 계층도 다양하여 흩어진 전부를 체계적으로 정리하는 것도 쉽지 않은 일이다.


In [68]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[7645])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

이 연구는 필자가 2010년 9월에 발표한  南明泉和尙頌證道歌 를 찍었던 고려활자와 관련된 계속 연구로 이미 알려져 있는 分司大藏都監에서 간행된  東國李相國集  全集의 版本도 그 底本은 證道歌字로 찍었다는 사실을 밝히는 내용이다.

그 동안 새로 발견된 고려 금속활자에 대한 다방면에서의 기초적 검토는 이미 발표되었으며 2010년  書誌學報 에 수록된 필자의  證道歌字의 발견과  南明泉和尙頌證道歌 의 연구 라는 논문에서 활자의 형태적 특징, 서체 및 탄소연대측정에 대한 분석 결과를 이미 언급한 바 있다.

또한 필자는 2004년부터 이 활자로 찍은 印本에 대하여 조사하면서 그동안 활자본의 飜刻 여부에 대하여 계속적인 의문을 가지고 있었던  東國李相國集 을 다시 검토하고 그 과정에서 수년 전에 번각본의 가능성을 제시하였던 윤병태 교수님의 선행연구를 살펴볼 수 있었다.

지금까지 각 연구영역에서 다루어졌던  東國李相國集 은 주로 역사학, 문학, 불교학 등에서 저자인 李奎報에 대한 傳記的 내용과 함께 기본자료로서 많이 이용되고 연구되어 왔다.

또 書誌的인 측면에서도 刊行과 版本에 대하여 부분적으로 다루어진 연구도 있었다.

그러나 초기의 판본들 중에는 補刻板의 混入이 많고 그 계층도 다양하여 흩어진 전부를 체계적으로 정리하는 것도 쉽지 않은 일이다.



In [69]:
print(preprocessing_sentences)

['이 연구는 필자가 2010년 9월에 발표한  南明泉和尙頌證道歌 를 찍었던 고려활자와 관련된 계속 연구로 이미 알려져 있는 分司大藏都監에서 간행된  東國李相國集  全集의 版本도 그 底本은 證道歌字로 찍었다는 사실을 밝히는 내용이다.', '그 동안 새로 발견된 고려 금속활자에 대한 다방면에서의 기초적 검토는 이미 발표되었으며 2010년  書誌學報 에 수록된 필자의  證道歌字의 발견과  南明泉和尙頌證道歌 의 연구 라는 논문에서 활자의 형태적 특징, 서체 및 탄소연대측정에 대한 분석 결과를 이미 언급한 바 있다.', '또한 필자는 2004년부터 이 활자로 찍은 印本에 대하여 조사하면서 그동안 활자본의 飜刻 여부에 대하여 계속적인 의문을 가지고 있었던  東國李相國集 을 다시 검토하고 그 과정에서 수년 전에 번각본의 가능성을 제시하였던 윤병태 교수님의 선행연구를 살펴볼 수 있었다.', '지금까지 각 연구영역에서 다루어졌던  東國李相國集 은 주로 역사학, 문학, 불교학 등에서 저자인 李奎報에 대한 傳記的 내용과 함께 기본자료로서 많이 이용되고 연구되어 왔다.', '또 書誌的인 측면에서도 刊行과 版本에 대하여 부분적으로 다루어진 연구도 있었다.', '그러나 초기의 판본들 중에는 補刻板의 混入이 많고 그 계층도 다양하여 흩어진 전부를 체계적으로 정리하는 것도 쉽지 않은 일이다.']


### AIHUB 요약문 및 레포트 생성 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=582)

In [70]:
path_list = ['AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/07.public/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/10.narration/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/07.public/' + '**/*.json',             
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/10.narration/' + '**/*.json']

train_file_name_01 = glob(path_list[0], recursive = True)
train_file_name_02 = glob(path_list[1], recursive = True)
train_file_name_03 = glob(path_list[2], recursive = True)
train_file_name_04 = glob(path_list[3], recursive = True)
train_file_name_05 = glob(path_list[4], recursive = True)
train_file_name_06 = glob(path_list[5], recursive = True)
train_file_name_07 = glob(path_list[6], recursive = True)
train_file_name_08 = glob(path_list[7], recursive = True)
train_file_name_09 = glob(path_list[8], recursive = True)
train_file_name_10 = glob(path_list[9], recursive = True)
valid_file_name_01 = glob(path_list[10], recursive = True)
valid_file_name_02 = glob(path_list[11], recursive = True)
valid_file_name_03 = glob(path_list[12], recursive = True)
valid_file_name_04 = glob(path_list[13], recursive = True)
valid_file_name_05 = glob(path_list[14], recursive = True)
valid_file_name_06 = glob(path_list[15], recursive = True)
valid_file_name_07 = glob(path_list[16], recursive = True)
valid_file_name_08 = glob(path_list[17], recursive = True)
valid_file_name_09 = glob(path_list[18], recursive = True)
valid_file_name_10 = glob(path_list[19], recursive = True)  

In [143]:
for train_file in train_file_name_09:
    with open(train_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)
        
        if '문서방' == one_json_sample['Meta(Acqusition)']['doc_name']:
            print(train_file)
            

AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\20per\REPORT-literature-07847-07847.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\20per\REPORT-literature-07877-07877.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\20per\REPORT-literature-07999-07999.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\20per\REPORT-literature-08130-08130.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\20per\REPORT-literature-08292-08292.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\20per\REPORT-literature-09328-09328.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\2~3sent\REPORT-literature-01200-01200.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\2~3sent\REPORT-literature-01513-01513.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\2~3sent\REPORT-literature-01990-01990.json
AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\2~3sent\REPORT-literature-02564-02564.json
AIHUB_요약문 및 레포트 생성 데이터

In [145]:
num = -1
for train_file in train_file_name_09:
    num += 1
    if "REPORT-literature-02564-02564.json" in train_file:
        print(num)

6164


In [147]:
source_file = train_file_name_09[6164]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source = one_json_sample['Meta(Refine)']['passage']

AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\2~3sent\REPORT-literature-02564-02564.json


In [148]:
print(source)

 그날도 무슨 말 끝에 짚두 없는데 가마닌 그렇게 많이 쳐서 뭣을 하느니, 만성이네는 쉰 장 배당인데 반만 치고 안 친다느니 그런 이야기가 벌어졌었 다. “허, 그래서 쓰나!” 문 서방은 바디를 쉬고 펄쩍뛰 었다. “걔가 학굘 좀 다니더니 너무 아는 체하나보드라. 없는 짚에 바쁜 백성들 한테 가마닐 치울 젠 나라에서두 쓸 데가 있어 그러겠지, 공연히 백성들 들 볶느라구 그럴까.” 그는 이렇게도 말했다. “사람이란 남의 공을 알어야 하느니라. 부모의 공두 알어야 하구, 이웃집 사람의 공두 알어야 하구, 나라 공두 알어야 하구. 바른 대루 말이지만 지 난해 삼 년이나 내리 흉년이 들었을 제 나라에서 그처럼 해주잖었으믄 이 근동만 해두 수백 명 굶어 죽었으리라. 뭐 벼 한 톨 있었다든? 너들은 모르 겠지만 옛날엔 흉년이 들면 그대루 앉아서 굶어 죽었느니라. 있는 놈두 못견 디어났지! 생각하면 지금 세상은 고마우니라. 연전 을축년 장마 때만 해두 몇 만 명이 굶어 죽은 줄 아니? 그런 공을 모르구 가마 좀 짜란다구 이러구 저러구 해? 몹쓸 생각이니라. 나라 공을 알어야지. 고마운 줄 알어야지. 만 성이 그놈 잘못 생각이지.” “주 성네보다두 많이 돌아갔나봐유.” 하고 중식이가 미처 말을 마치기도 전에 문 서방은 말꼬리를 툭 채서, “거 다 못된 생각이지. 그런 일거리가 아니구 나라에서 모찌떡을 남보다 더 줬대두 그 녀석 투정을 할까? 도시 그 애가 못쓸 레라. 제 부모 은혤 모르 는 놈이니 나라 공을 알랴만서두…” 이런 문 서방이다.



In [149]:
preprocessing_sentences = formal_preprocessing_text_kss3(source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

그날도 무슨 말 끝에 짚두 없는데 가마닌 그렇게 많이 쳐서 뭣을 하느니, 만성이네는 쉰 장 배당인데 반만 치고 안 친다느니 그런 이야기가 벌어졌었 허, 그래서 쓰나 문 서방은 바디를 쉬고 펄쩍뛰 었다.

걔가 학굘 좀 다니더니 너무 아는 체하나보드라.

없는 짚에 바쁜 백성들 한테 가마닐 치울 젠 나라에서두 쓸 데가 있어 그러겠지, 공연히 백성들 들 볶느라구 그럴까.

그는 이렇게도 말했다.

사람이란 남의 공을 알어야 하느니라.

부모의 공두 알어야 하구, 이웃집 사람의 공두 알어야 하구, 나라 공두 알어야 하구. 바른 대루 말이지만 지 난해 삼 년이나 내리 흉년이 들었을 제 나라에서 그처럼 해주잖었으믄 이 근동만 해두 수백 명 굶어 죽었으리라.

뭐 벼 한 톨 있었다든 너들은 모르 겠지만 옛날엔 흉년이 들면 그대루 앉아서 굶어 죽었느니라.

연전 을축년 장마 때만 해두 몇 만 명이 굶어 죽은 줄 아니 그런 공을 모르구 가마 좀 짜란다구 이러구 저러구 해 몹쓸 생각이니라.

만 성이 그놈 잘못 생각이지.

주 성네보다두 많이 돌아갔나봐유. 하고 중식이가 미처 말을 마치기도 전에 문 서방은 말꼬리를 툭 채서, 거 다 못된 생각이지.

그런 일거리가 아니구 나라에서 모찌떡을 남보다 더 줬대두 그 녀석 투정을 할까

제 부모 은혤 모르 는 놈이니 나라 공을 알랴만서두 이런 문 서방이다.



In [150]:
print(preprocessing_sentences)

['그날도 무슨 말 끝에 짚두 없는데 가마닌 그렇게 많이 쳐서 뭣을 하느니, 만성이네는 쉰 장 배당인데 반만 치고 안 친다느니 그런 이야기가 벌어졌었 허, 그래서 쓰나 문 서방은 바디를 쉬고 펄쩍뛰 었다.', '걔가 학굘 좀 다니더니 너무 아는 체하나보드라.', '없는 짚에 바쁜 백성들 한테 가마닐 치울 젠 나라에서두 쓸 데가 있어 그러겠지, 공연히 백성들 들 볶느라구 그럴까.', '그는 이렇게도 말했다.', '사람이란 남의 공을 알어야 하느니라.', '부모의 공두 알어야 하구, 이웃집 사람의 공두 알어야 하구, 나라 공두 알어야 하구. 바른 대루 말이지만 지 난해 삼 년이나 내리 흉년이 들었을 제 나라에서 그처럼 해주잖었으믄 이 근동만 해두 수백 명 굶어 죽었으리라.', '뭐 벼 한 톨 있었다든 너들은 모르 겠지만 옛날엔 흉년이 들면 그대루 앉아서 굶어 죽었느니라.', '연전 을축년 장마 때만 해두 몇 만 명이 굶어 죽은 줄 아니 그런 공을 모르구 가마 좀 짜란다구 이러구 저러구 해 몹쓸 생각이니라.', '만 성이 그놈 잘못 생각이지.', '주 성네보다두 많이 돌아갔나봐유. 하고 중식이가 미처 말을 마치기도 전에 문 서방은 말꼬리를 툭 채서, 거 다 못된 생각이지.', '그런 일거리가 아니구 나라에서 모찌떡을 남보다 더 줬대두 그 녀석 투정을 할까', '제 부모 은혤 모르 는 놈이니 나라 공을 알랴만서두 이런 문 서방이다.']


### AIHUB 문서요약 텍스트

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97)

In [293]:
path_list = ['AIHUB_문서요약 텍스트/Training/'+ '/**/*.json', 
             'AIHUB_문서요약 텍스트/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [295]:
source_file = train_file_name[2]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
source_df = pd.DataFrame(one_json_sample['documents'])
source_dict = dict(source_df['text'].explode())
source_json = pd.json_normalize(source_dict)  

source_list = []
for source_index in source_json:
    for source_nested_list in source_json[source_index]:
        for source_single_list in source_nested_list:
            source_sentence = ""
            for source_single in source_single_list:
                if type(source_single) == dict:
                    for key, value in source_single.items():
                        if key == "sentence":
                            source_sentence += " " + value
            if len(source_sentence) > 0:
                source_list.append(source_sentence)

AIHUB_문서요약 텍스트/Training\신문기사_train_original\train_original.json


In [ ]:
for i in range(0, len(source_list)):
    if '#' in source_list[i]:
        print(source_list[i])
        print(i)

In [ ]:
bracket_form = re.compile('\(([^)]+)')

for i in range(0, len(source_list)):
    
    text_in_small_bracket = bracket_form.findall(source_list[i])
    try:
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source_list[i]) >= source_list[i] and source_list[i][last_index-text_size-1] == '(' and source_list[i][last_index+1] == '.':
                source_list[i]= source_list[i].replace(source_list[i][last_index-text_size-1 : last_index+1] + ".", ".")
                print(source_list[i])
                print("\n")
                print("[After]")
                print(source)
                print(i)
                print("\n")


            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source_list[i].replace(small_bracket, " " + text + " ") 
                print(source_list[i])
                print("\n")
                print("[After]")
                print(source)
                print(i)
                print("\n")

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source_list[i].find(text) + len(text)
                if len(source_list[i]) >= last_index+1 and source_list[i][last_index-text_size-1] == '(' and source_list[i][last_index+1] == '.':
                    source = source_list[i].replace(source_list[i][last_index-text_size-1 : last_index+1] + ".", ".")
                    print(source_list[i])
                    print("\n")
                    print("[After]")
                    print(source)
                    print(i)
                    print("\n")


                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source_list[i].replace(small_bracket, " " + text + " ") 
                    print(source_list[i])
                    print("\n")
                    print("[After]")
                    print(source)
                    print(i)
                    print("\n")
                
    except:
            pass

In [183]:
num = -1
for source in source_list:
    num += 1
    if "무주공산이 된 이 행성에 뜻하지 않은 일이 벌어진다" in source:
        print(num)

66842


In [726]:
source_json[0][0][0]['sentence']

'원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,'

In [18]:
for i in range(len(source_json)):
    for j in range(len(source_json[i])):
        for k in range(len(source_json[i][j])):
            if '무주공산' in source_json[i][j][k]['sentence']:
                print("i: ", i)
                print("j: ", j)
                print("j: ", k)

In [184]:
print(source_list[66842])

 그 결과 무주공산이 된 이 행성에 뜻하지 않은 일이 벌어진다. 바로 토종동물들의 멸종이다. 이들은 모두 느림보 초식동물들이라 지구에서 온 포식자들에게 죄다 먹힌 것이다.(왜 미네르바의 동물들이 죄다 초식성인지는 조금만 기다리시라.) 지구동물(특히 육식동물)들이 미네르바에 와서 지능이 비상하게 발달한 까닭이다.


In [185]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[66842])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

그 결과 무주공산이 된 이 행성에 뜻하지 않은 일이 벌어진다.

이들은 모두 느림보 초식동물들이라 지구에서 온 포식자들에게 죄다 먹힌 것이다.

왜 미네르바의 동물들이 죄다 초식성인지는 조금만 기다리시라.

지구동물(특히 육식동물)들이 미네르바에 와서 지능이 비상하게 발달한 까닭이다.



In [186]:
print(preprocessing_sentences)

['그 결과 무주공산이 된 이 행성에 뜻하지 않은 일이 벌어진다.', '이들은 모두 느림보 초식동물들이라 지구에서 온 포식자들에게 죄다 먹힌 것이다.', '왜 미네르바의 동물들이 죄다 초식성인지는 조금만 기다리시라.', '지구동물(특히 육식동물)들이 미네르바에 와서 지능이 비상하게 발달한 까닭이다.']


### AIHUB 특허분야 자동분류 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=116&topMenu=100&aihubDataSe=ty&dataSetSn=547)

In [187]:
path_list = ['AIHUB_특허 분야 자동분류 데이터/Training/'+ '/**/*.json', 
             'AIHUB_특허 분야 자동분류 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [285]:
num = -1
for train_file in train_file_name:
    num += 1
    if "석유화학계기초화학물질제조업" in train_file:
        print(num)

467


In [286]:
source_file = train_file_name[467]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_list = list(pd.DataFrame(one_json_sample['dataset'])['abstract'].dropna())

AIHUB_특허 분야 자동분류 데이터/Training\원천데이터\TS1\C_제조업_10_34\20_화학물질및화학제품제조업;의약품제외\201_기초화학물질제조업\20111_석유화학계기초화학물질제조업.json


In [ ]:
num = 80

for i in range(0, len(source_list)):
    if '+' in source_list[i]:
        try : 
            preprocessing_sentences = formal_preprocessing_text_kss1(source_list[i])
            if type(preprocessing_sentences) == list and len(preprocessing_sentences) > 1 and preprocessing_sentences[0] != source_list[i]:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
        except:
            pass

In [297]:
num = -1
for source in source_list:
    num += 1
    if "본 발명은 수소화처리 프로세스" in source:
        print(num)

514


In [298]:
print(source_list[514])

본 발명은 수소화처리 프로세스 또는 수소화분해 프로세스를 위해서 의도된 몰리브덴에 기반된 촉매의 제조방법에 관한 것이다.본 발명은 수소화처리 프로세스 및 수소화분해 프로세스에서 수소화 반응들을 실시하기 위한 촉매의 제조방법에 관한 것이다. 상기 촉매는, 적어도 하나의 Mo=O 또는 Mo-OR 결합 또는 적어도 하나의 Mo=S 또는 Mo-SR 결합 [R = CxHy, x ≥ 1 및 ( x - 1 ) ≤ y ≤ ( 2x + 1 ) 또는 R = Si (OR')3 또는 R = Si(R')3, 여기서 R' = Cx'Hy', x' ≥ 1 및 ( x' - 1 ) ≤ y' ≤ ( 2x' + 1 )] 을 갖고, 단위체 또는 이합체의 형태로 몰리브덴에 기반된 적어도 하나의 단핵 전구체로부터, 그리고 선택적으로 VIII 족으로부터의 적어도 하나의 촉진제 원소로부터 제조된다. 상기 전구체들은 산화물 담지체 상에 증착되며, 이 담지체는 이것이 사용되는 프로세스에 적합하고, 상기 촉매는 200 ℃ 미만의 온도에서 건조되고, 다음으로, 유리하게는 상기 프로세스에 배치되기 전에 황화된다.


In [300]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[514])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

본 발명은 수소화처리 프로세스 또는 수소화분해 프로세스를 위해서 의도된 몰리브덴에 기반된 촉매의 제조방법에 관한 것이다.

본 발명은 수소화처리 프로세스 및 수소화분해 프로세스에서 수소화 반응들을 실시하기 위한 촉매의 제조방법에 관한 것이다.

상기 촉매는, 적어도 하나의 Mo O 또는 Mo-OR 결합 또는 적어도 하나의 Mo S 또는 Mo-SR 결합 을 갖고, 단위체 또는 이합체의 형태로 몰리브덴에 기반된 적어도 하나의 단핵 전구체로부터, 그리고 선택적으로 VIII 족으로부터의 적어도 하나의 촉진제 원소로부터 제조된다.

상기 전구체들은 산화물 담지체 상에 증착되며, 이 담지체는 이것이 사용되는 프로세스에 적합하고, 상기 촉매는 200   미만의 온도에서 건조되고, 다음으로, 유리하게는 상기 프로세스에 배치되기 전에 황화된다.



In [301]:
print(preprocessing_sentences)

['본 발명은 수소화처리 프로세스 또는 수소화분해 프로세스를 위해서 의도된 몰리브덴에 기반된 촉매의 제조방법에 관한 것이다.', '본 발명은 수소화처리 프로세스 및 수소화분해 프로세스에서 수소화 반응들을 실시하기 위한 촉매의 제조방법에 관한 것이다.', '상기 촉매는, 적어도 하나의 Mo O 또는 Mo-OR 결합 또는 적어도 하나의 Mo S 또는 Mo-SR 결합 을 갖고, 단위체 또는 이합체의 형태로 몰리브덴에 기반된 적어도 하나의 단핵 전구체로부터, 그리고 선택적으로 VIII 족으로부터의 적어도 하나의 촉진제 원소로부터 제조된다.', '상기 전구체들은 산화물 담지체 상에 증착되며, 이 담지체는 이것이 사용되는 프로세스에 적합하고, 상기 촉매는 200   미만의 온도에서 건조되고, 다음으로, 유리하게는 상기 프로세스에 배치되기 전에 황화된다.']


### AIHUB 뉴스기사 기계독해 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=577)

In [11]:
path_list = ['AIHUB_뉴스 기사 기계독해 데이터/Training/'+ '/**/*.json', 
             'AIHUB_뉴스 기사 기계독해 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [20]:
source_file = train_file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_뉴스 기사 기계독해 데이터/Training\원천데이터\TS_unanswerable.json


In [ ]:
num = 80

for i in range(0, num):
    if '“' in source_list[i]:
        try : 
            preprocessing_sentences = formal_preprocessing_text_kss1(source_list[i])
            if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
                print("[Before]")
                print(source_list[i])
                print("\n")

                print("[After]")
                for preprocessing_sentence in preprocessing_sentences:          
                        print(preprocessing_sentence, end="\n")
                        print("\n")
        except:
            pass

In [34]:
num = -1
for source in source_list:
    num += 1
    if "시스템 반도체 상생펀드" in source:
        print(num)

52


In [35]:
print(source_list[52])

홍남기 경제부총리 겸 기획재정부 장관은 친환경차의 획기적 보급을 위해 도시공원과 그린벨트 내에 수소충전소 설치를 허용하겠다고 밝혔다. 
 
홍 부총리는 25일 정부서울청사에서 열린 제5차 혁신성장 BIG3 추진회의에서 “올해 친환경차 30만대 시대 목표달성을 위해 충전·이용·주차 중심 10대 과제를 연내 중점 개혁해나갈 것”이라며 이같이 말했다. 정부는 사용자 편의를 높이기 위해 전기차 충전시설 설치 시 노외주차장 20% 이내이던 면적 제한을 없애고 공공 충전기를 의무 개방하기로 했다. 
 
현행 자동차정비업 등록 기준이 내연차 기준으로 돼 있어 전기차 전문정비소의 경우 불필요한 시설장비 구비 부담이 있는 점을 고려해 내연차 정비에 필요한 장비는 갖추지 않아도 등록할 수 있도록 시설기준도 완화한다. 
 
편리한 주차를 위해 친환경차 전용 주차구역을 노외·공공건물의 5% 이상으로 확대하고, 완속충전구역 사용을 최대 12시간까지만 허용하는 등 장시간 점유에 따른 주차불편도 해소하기로 했다. 홍 부총리는 “이를 뒷받침하기 위해 관련법, 시행령, 시행규칙 개정을 최대한 조속히 추진하겠다”고 말했다. 
 
시스템 반도체와 관련해서는 민간투자를 가속화하고 투자·보증을 강화하기로 했다. 홍 부총리는 “대규모 투자가 예정된 용인 반도체 클러스터는 수도권정비위원회 및 산단계획 승인 등 절차를 3월 중 마무리하고 용수와 전력 등 기반시설이 적기에 구축되도록 지원하겠다”고 말했다. 
 
그는 “시스템 반도체 상생펀드 1000억원, 성장펀드 2000억원, ‘DNA+Big3’ 펀드 2500억원, 소재·부품·장비 반도체 펀드 1000억원 등 총 6400억원 이상의 펀드 조성·지원을 추진하겠다”며 “올해는 소재·부품·장비 반도체 펀드 1000억원 신규 조성 등 2800억원을 조성할 방침”이라고 강조했다. 이어 “보증 심사 시 영세하지만 우수한 기술을 보유한 팹리스 업체에 대해서는 산업구조고도화 프로그램, 기술보증기금·신용보증기금 특화 보증 프로그램 등 지원을 강화할 것”이라고 덧붙였다.

In [36]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[52])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

홍남기 경제부총리 겸 기획재정부 장관은 친환경차의 획기적 보급을 위해 도시공원과 그린벨트 내에 수소충전소 설치를 허용하겠다고 밝혔다.

홍 부총리는 25일 정부서울청사에서 열린 제5차 혁신성장 BIG3 추진회의에서  올해 친환경차 30만대 시대 목표달성을 위해 충전 이용 주차 중심 10대 과제를 연내 중점 개혁해나갈 것 이라며 이같이 말했다.

정부는 사용자 편의를 높이기 위해 전기차 충전시설 설치 시 노외주차장 20  이내이던 면적 제한을 없애고 공공 충전기를 의무 개방하기로 했다.

현행 자동차정비업 등록 기준이 내연차 기준으로 돼 있어 전기차 전문정비소의 경우 불필요한 시설장비 구비 부담이 있는 점을 고려해 내연차 정비에 필요한 장비는 갖추지 않아도 등록할 수 있도록 시설기준도 완화한다.

편리한 주차를 위해 친환경차 전용 주차구역을 노외 공공건물의 5  이상으로 확대하고, 완속충전구역 사용을 최대 12시간까지만 허용하는 등 장시간 점유에 따른 주차불편도 해소하기로 했다.

홍 부총리는  이를 뒷받침하기 위해 관련법, 시행령, 시행규칙 개정을 최대한 조속히 추진하겠다 고 말했다.

시스템 반도체와 관련해서는 민간투자를 가속화하고 투자 보증을 강화하기로 했다.

홍 부총리는  대규모 투자가 예정된 용인 반도체 클러스터는 수도권정비위원회 및 산단계획 승인 등 절차를 3월 중 마무리하고 용수와 전력 등 기반시설이 적기에 구축되도록 지원하겠다 고 말했다.

그는  시스템 반도체 상생펀드 1000억원, 성장펀드 2000억원,  DNA+Big3  펀드 2500억원, 소재 부품 장비 반도체 펀드 1000억원 등 총 6400억원 이상의 펀드 조성 지원을 추진하겠다 며  올해는 소재 부품 장비 반도체 펀드 1000억원 신규 조성 등 2800억원을 조성할 방침 이라고 강조했다.

이어  보증 심사 시 영세하지만 우수한 기술을 보유한 팹리스 업체에 대해서는 산업구조고도화 프로그램, 기술보증기금 신용보증기금 특화 보증 프로그램 등 지원을 강화할 것 이라고 덧붙였다.

아울러 

In [37]:
print(preprocessing_sentences)

['홍남기 경제부총리 겸 기획재정부 장관은 친환경차의 획기적 보급을 위해 도시공원과 그린벨트 내에 수소충전소 설치를 허용하겠다고 밝혔다.', '홍 부총리는 25일 정부서울청사에서 열린 제5차 혁신성장 BIG3 추진회의에서  올해 친환경차 30만대 시대 목표달성을 위해 충전 이용 주차 중심 10대 과제를 연내 중점 개혁해나갈 것 이라며 이같이 말했다.', '정부는 사용자 편의를 높이기 위해 전기차 충전시설 설치 시 노외주차장 20  이내이던 면적 제한을 없애고 공공 충전기를 의무 개방하기로 했다.', '현행 자동차정비업 등록 기준이 내연차 기준으로 돼 있어 전기차 전문정비소의 경우 불필요한 시설장비 구비 부담이 있는 점을 고려해 내연차 정비에 필요한 장비는 갖추지 않아도 등록할 수 있도록 시설기준도 완화한다.', '편리한 주차를 위해 친환경차 전용 주차구역을 노외 공공건물의 5  이상으로 확대하고, 완속충전구역 사용을 최대 12시간까지만 허용하는 등 장시간 점유에 따른 주차불편도 해소하기로 했다.', '홍 부총리는  이를 뒷받침하기 위해 관련법, 시행령, 시행규칙 개정을 최대한 조속히 추진하겠다 고 말했다.', '시스템 반도체와 관련해서는 민간투자를 가속화하고 투자 보증을 강화하기로 했다.', '홍 부총리는  대규모 투자가 예정된 용인 반도체 클러스터는 수도권정비위원회 및 산단계획 승인 등 절차를 3월 중 마무리하고 용수와 전력 등 기반시설이 적기에 구축되도록 지원하겠다 고 말했다.', '그는  시스템 반도체 상생펀드 1000억원, 성장펀드 2000억원,  DNA+Big3  펀드 2500억원, 소재 부품 장비 반도체 펀드 1000억원 등 총 6400억원 이상의 펀드 조성 지원을 추진하겠다 며  올해는 소재 부품 장비 반도체 펀드 1000억원 신규 조성 등 2800억원을 조성할 방침 이라고 강조했다.', '이어  보증 심사 시 영세하지만 우수한 기술을 보유한 팹리스 업체에 대해서는 산업구조고도화 프로그램, 기술보증기금 신용보증기금 특화 보증 프로그램 등 지원을 강

### AIHUB 행정 문서 대상 기계독해 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=569)

In [121]:
path_list = ['AIHUB_행정 문서 대상 기계독해 데이터/Training/'+ '/**/*.json', 
             'AIHUB_행정 문서 대상 기계독해 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [122]:
source_file = train_file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file) 

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_tableqa.json


In [123]:
num = -1
for source in source_list:
    num += 1
    if "여름철 고압가스 충전시설 안전점검" in source:
        print(num)

42


In [124]:
print(source_list[42])

제  목 2021년 여름철 고압가스 충전시설 안전점검(자체) 계획 알림<br>1. 은평구청 환경과-18398(2021.6.22.)호『2021년 여름철 고압가스 충전시설 안전점검 결과 제출 요청』과 관련입니다.<br>2. 2021년 여름철 가스시설 안전점검 계획에 따라 고압가스 관련시설에 대한 자체안전점검 실시 후 결과를 2021.7.28.(수)까지 제출하여 주시기 바랍니다. <table><tbody><tr><td>연번</td><td>시설명</td><td>가스종류</td><td>소재지</td><td>비고</td></tr><tr><td>1</td><td>은평소방서(현장대응단)</td><td>공기압축(호흡기용 공기충전기)</td><td>통일로962</td><td></td></tr><tr><td>2</td><td>은평소방서(수색119안전센터)</td><td>공기압축(호흡기용 공기충전기)</td><td>수색로320</td><td></td></tr></tbody></table>


In [125]:
preprocessing_sentences = tableqa_preprocessing_text(source_list[42])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

1. 은평구청 환경과-18398(2021.6.22.)호 2021년 여름철 고압가스 충전시설 안전점검 결과 제출 요청 과 관련입니다.

2. 2021년 여름철 가스시설 안전점검 계획에 따라 고압가스 관련시설에 대한 자체안전점검 실시 후 결과를 2021.7.28.(수)까지 제출하여 주시기 바랍니다.



In [126]:
print(preprocessing_sentences)

['1. 은평구청 환경과-18398(2021.6.22.)호 2021년 여름철 고압가스 충전시설 안전점검 결과 제출 요청 과 관련입니다.', '2. 2021년 여름철 가스시설 안전점검 계획에 따라 고압가스 관련시설에 대한 자체안전점검 실시 후 결과를 2021.7.28.(수)까지 제출하여 주시기 바랍니다.']


In [127]:
source_file = train_file_name[2]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file) 

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_multiple_choice.json


In [128]:
num = -1
for source in source_list:
    num += 1
    if "신규공사장 안전기술지도 시행계획 변경" in source:
        print(num)

22


In [129]:
source_list[22]

'제목 2021년 6월 신규공사장 안전기술지도 시행계획 변경\n1. 2021년 본부장 신년 업무보고(2021.1.27.)와 관련하여 「신규공사장 안전기술지도」2021년 6월 시행계획을 다음과 같이 보고드립니다.\n 가. 지도일정 : 2021년 6월 10일(목), 29일(화) 오전\n 나. 지도강사 : 시설국장, 안전관리과장, 안전관리과 정진혁, 이태행\n 다. 교육시간 : 오전 09:30 ~ 12:30(3.0H)\n 라. 교육장소 : 도시기반시설본부 10층 회의실\n 마. 교육대상 : 현장소장, 감리단장, 안전관리자 등\n 바. 교육내용\n 1) 건설공사장 안전관리 방향\n 2) 건설기술진흥법, 산업안전보건법 상의 안전관리 내용과 구비 및 관리 서류(1.5H)\n 3) 도시기반시설본부 건설안전 관련 각종 방침 및 지침'

In [130]:
print(source_list[22])

제목 2021년 6월 신규공사장 안전기술지도 시행계획 변경
1. 2021년 본부장 신년 업무보고(2021.1.27.)와 관련하여 「신규공사장 안전기술지도」2021년 6월 시행계획을 다음과 같이 보고드립니다.
 가. 지도일정 : 2021년 6월 10일(목), 29일(화) 오전
 나. 지도강사 : 시설국장, 안전관리과장, 안전관리과 정진혁, 이태행
 다. 교육시간 : 오전 09:30 ~ 12:30(3.0H)
 라. 교육장소 : 도시기반시설본부 10층 회의실
 마. 교육대상 : 현장소장, 감리단장, 안전관리자 등
 바. 교육내용
 1) 건설공사장 안전관리 방향
 2) 건설기술진흥법, 산업안전보건법 상의 안전관리 내용과 구비 및 관리 서류(1.5H)
 3) 도시기반시설본부 건설안전 관련 각종 방침 및 지침


In [131]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[22])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

1. 2021년 본부장 신년 업무보고(2021.1.27.)와 관련하여  신규공사장 안전기술지도 2021년 6월 시행계획을 다음과 같이 보고드립니다.



In [132]:
print(preprocessing_sentences)

['1. 2021년 본부장 신년 업무보고(2021.1.27.)와 관련하여  신규공사장 안전기술지도 2021년 6월 시행계획을 다음과 같이 보고드립니다.']


### AIHUB 기계독해

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=89)

In [20]:
path_list = ['AIHUB_기계독해'+ '/**/*.json']
file_name = json_file_name_list(path_list)

In [135]:
source_file = file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_기계독해\ko_nia_normal_squad_all.json


In [ ]:
num = 130

for i in range(num-10, num):
    try : 
        preprocessing_sentences = formal_preprocessing_text_kss1(source_list[i])
        if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
            print("[Before]")
            print(source_list[i])
            print("\n")

            print("[After]")
            for preprocessing_sentence in preprocessing_sentences:          
                    print(preprocessing_sentence, end="\n")
                    print("\n")
    except:
        pass

In [140]:
print(source_list[123])

[IMG1][이슈데일리 최준용기자]민경훈이 정규 2집 앨범 ‘소풍’을 통해 1년 4개월 만에 가요계에 컴백했다. 민경훈의 ‘소풍’은 27일 자정, 전 온라인 음원 사이트를 통해 공개됐다. 이번에 공개된 앨범에는 총 10곡 이 수록됐으며, 각 각의 곡 모두 다른 분위기와 창법으로 차별화를 시도했다. 특히 민경훈은 이번 앨범에서 직접 프로듀싱에 나섰으며, 자작곡은 물론 유명 작곡가 김도훈, 전해성, 김세진, 이창현, 더 네임, 강우현, 오승은, 스웨덴의 작곡가 Hal Marabel, Jonas Reingold 등이 참여해 완성도를 높였다. 타이틀곡인 ‘쉬(She)’는 김도훈과 최갑원이 작곡, 작사한 곡으로 여름과 어울리는 시원하고 빠른 템포의 곡이다. 또 이 곡은 사랑하는 사람에 대한 고백송으로 민경훈의 감미롭고도 시원한 보컬이 가사를 잘 표현해주고 있다. 또 이번 앨범에는 SG워너비 이석훈이 함께한 듀엣곡을 수록돼 눈길을 끈다. 한편 민경훈은 오는 30일 MNET ‘엠카운트다운’을 통해 컴백 무대를 가질 예정이다.


In [141]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[123])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

민경훈이 정규 2집 앨범  소풍 을 통해 1년 4개월 만에 가요계에 컴백했다.

민경훈의  소풍 은 27일 자정, 전 온라인 음원 사이트를 통해 공개됐다.

이번에 공개된 앨범에는 총 10곡 이 수록됐으며, 각 각의 곡 모두 다른 분위기와 창법으로 차별화를 시도했다.

특히 민경훈은 이번 앨범에서 직접 프로듀싱에 나섰으며, 자작곡은 물론 유명 작곡가 김도훈, 전해성, 김세진, 이창현, 더 네임, 강우현, 오승은, 스웨덴의 작곡가 Hal Marabel, Jonas Reingold 등이 참여해 완성도를 높였다.

타이틀곡인  쉬(She) 는 김도훈과 최갑원이 작곡, 작사한 곡으로 여름과 어울리는 시원하고 빠른 템포의 곡이다.

또 이 곡은 사랑하는 사람에 대한 고백송으로 민경훈의 감미롭고도 시원한 보컬이 가사를 잘 표현해주고 있다.

또 이번 앨범에는 SG워너비 이석훈이 함께한 듀엣곡을 수록돼 눈길을 끈다.

한편 민경훈은 오는 30일 MNET  엠카운트다운 을 통해 컴백 무대를 가질 예정이다.



In [143]:
print(preprocessing_sentences)

['민경훈이 정규 2집 앨범  소풍 을 통해 1년 4개월 만에 가요계에 컴백했다.', '민경훈의  소풍 은 27일 자정, 전 온라인 음원 사이트를 통해 공개됐다.', '이번에 공개된 앨범에는 총 10곡 이 수록됐으며, 각 각의 곡 모두 다른 분위기와 창법으로 차별화를 시도했다.', '특히 민경훈은 이번 앨범에서 직접 프로듀싱에 나섰으며, 자작곡은 물론 유명 작곡가 김도훈, 전해성, 김세진, 이창현, 더 네임, 강우현, 오승은, 스웨덴의 작곡가 Hal Marabel, Jonas Reingold 등이 참여해 완성도를 높였다.', '타이틀곡인  쉬(She) 는 김도훈과 최갑원이 작곡, 작사한 곡으로 여름과 어울리는 시원하고 빠른 템포의 곡이다.', '또 이 곡은 사랑하는 사람에 대한 고백송으로 민경훈의 감미롭고도 시원한 보컬이 가사를 잘 표현해주고 있다.', '또 이번 앨범에는 SG워너비 이석훈이 함께한 듀엣곡을 수록돼 눈길을 끈다.', '한편 민경훈은 오는 30일 MNET  엠카운트다운 을 통해 컴백 무대를 가질 예정이다.']


In [21]:
source_file = file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

AIHUB_기계독해\ko_nia_normal_squad_all.json


In [ ]:
source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

### AIHUB 도서자료 요약

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=93)

In [106]:
path_list = ['AIHUB_도서자료 요약/Training/'+ '/**/*.json', 
             'AIHUB_도서자료 요약/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [110]:
train_file_name.sort()

In [111]:
num = -1
for source in source_list:
    num += 1
    if "여름철 고압가스 충전시설 안전점검" in source:
        print(num)

In [112]:
count1 = sum(map(lambda x : '기술과학' in x, train_file_name))
count2 = sum(map(lambda x : '기타' in x, train_file_name))
count3 = sum(map(lambda x : '사회과학' in x, train_file_name))
count4 = sum(map(lambda x : '예술' in x, train_file_name))
science = count1
etc = count1 + count2
society = count1 + count2 + count3
art = count1 + count2 + count3 + count4
print(science,  etc, society, art)

23918 30671 146111 160002


In [287]:
source_file = train_file_name[10090]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

passage_list = []
summary_list = []
passage = one_json_sample["passage"]  
summary = one_json_sample["summary"]

passage_sentences = formal_preprocessing_text_kss3(passage)
for sentence in passage_sentences:
    passage_list.append(sentence)   

summary_sentences = formal_preprocessing_text_kss3(summary)
for sentence in summary_sentences:
    summary_list.append(sentence) 

AIHUB_도서자료 요약/Training\[원천]도서요약_train\기술과학\PCY_201611170453294911_77.json


In [291]:
passage

'그리고 정부는 유전공학육성법과 시행령이 제정된 직후인 1985년 2월 1일 한국과학기술원 산하에 유전공학센터를 설립했다. 당시 한국과학기술원에 재직 중이던 한문희 교수가 초대 소장을 맡은 유전공학센터는 소장을 중심으로 연구업무를 수행하는 연구부와 ‘유전자은행 설립, DNA 안정성 실험 및 효소․숙주 등 기본소재의 생산과 공급’을 관할하는 특수사업부, 그리고 중장기 연구계획을 마련하고 산학과의 연계 및 사무업무를 담당할 연구기획실로 구성되었다. 특히 연구부에는 6개의 연구실을 두고, 각각 효소공학, 분자생물학, 미생물공학, 미생물유전자, 세포분양, 생물고분자화학 등을 연구하도록 했다. 설립 당시 한국과학기술원의 기존 연구 인력과 교수진 40여 명을 바탕으로 설립된 유전공학센터는 재외 한국인 과학자를 적극 유치할 전망도 제시했다(경향신문, 1985. 2. 5; 매일경제, 1985. 2. 7). 1986년 10월 유전공학센터는 기존 체계를 2부 15연구실 1연구기획실로 확대 개편하면서 식량시스템연구실, 생물화학공정연구실, 단백질화학연구실, 미생물생태연구실 등을 신설했고(매일경제, 1986. 10. 9), 1987년에는 60명의 박사학위 연구원을 포함 166명의 직원이 종사하는 센터로 커졌다(한국생화학분자생물학회, 1987). [그림 2-2]는 1987년 당시 유전공학센터의 조직도이다.'

In [288]:
print(passage)

그리고 정부는 유전공학육성법과 시행령이 제정된 직후인 1985년 2월 1일 한국과학기술원 산하에 유전공학센터를 설립했다. 당시 한국과학기술원에 재직 중이던 한문희 교수가 초대 소장을 맡은 유전공학센터는 소장을 중심으로 연구업무를 수행하는 연구부와 ‘유전자은행 설립, DNA 안정성 실험 및 효소․숙주 등 기본소재의 생산과 공급’을 관할하는 특수사업부, 그리고 중장기 연구계획을 마련하고 산학과의 연계 및 사무업무를 담당할 연구기획실로 구성되었다. 특히 연구부에는 6개의 연구실을 두고, 각각 효소공학, 분자생물학, 미생물공학, 미생물유전자, 세포분양, 생물고분자화학 등을 연구하도록 했다. 설립 당시 한국과학기술원의 기존 연구 인력과 교수진 40여 명을 바탕으로 설립된 유전공학센터는 재외 한국인 과학자를 적극 유치할 전망도 제시했다(경향신문, 1985. 2. 5; 매일경제, 1985. 2. 7). 1986년 10월 유전공학센터는 기존 체계를 2부 15연구실 1연구기획실로 확대 개편하면서 식량시스템연구실, 생물화학공정연구실, 단백질화학연구실, 미생물생태연구실 등을 신설했고(매일경제, 1986. 10. 9), 1987년에는 60명의 박사학위 연구원을 포함 166명의 직원이 종사하는 센터로 커졌다(한국생화학분자생물학회, 1987). [그림 2-2]는 1987년 당시 유전공학센터의 조직도이다.


In [289]:
print(passage_list)

['그리고 정부는 유전공학육성법과 시행령이 제정된 직후인 1985년 2월 1일 한국과학기술원 산하에 유전공학센터를 설립했다.', '당시 한국과학기술원에 재직 중이던 한문희 교수가 초대 소장을 맡은 유전공학센터는 소장을 중심으로 연구업무를 수행하는 연구부와 유전자은행 설립, DNA 안정성 실험 및 효소 숙주 등 기본소재의 생산과 공급 을 관할하는 특수사업부, 그리고 중장기 연구계획을 마련하고 산학과의 연계 및 사무업무를 담당할 연구기획실로 구성되었다.', '특히 연구부에는 6개의 연구실을 두고, 각각 효소공학, 분자생물학, 미생물공학, 미생물유전자, 세포분양, 생물고분자화학 등을 연구하도록 했다.', '설립 당시 한국과학기술원의 기존 연구 인력과 교수진 40여 명을 바탕으로 설립된 유전공학센터는 재외 한국인 과학자를 적극 유치할 전망도 제시했다.', '1986년 10월 유전공학센터는 기존 체계를 2부 15연구실 1연구기획실로 확대 개편하면서 식량시스템연구실, 생물화학공정연구실, 단백질화학연구실, 미생물생태연구실 등을 신설했고(매일경제, 1986. 10. 9), 1987년에는 60명의 박사학위 연구원을 포함 166명의 직원이 종사하는 센터로 커졌다.']


In [290]:
print(summary)

정부는 1985년 한국과학기술원 산하에 유전공학센터를 설립하였다. 초기 센터는 40여 명을 바탕으로 연구부, 특수사업부, 연구기획실로 구성되어 효소공학, 분자 생물학, 세포분양 등을 연구하였다. 센터는 인재의 적극 스카웃과 연구실 확대 개편으로 연구 분야를 다양화시켰고 그 결과 1987년 유전공학센터는 166명이 종사하는 센터로 확대되였다.


In [231]:
print(summary_list)

['새로운 방식으로 부의 창출을 촉진시키는 디지털 기술 중 IoT가 주목받고 있다.', '이것은 다양한 산업들의 서비스와 동반성장할 것이다.', '또한, 디지털 기술 중 블록체인 시스템을 통해 안면 없는 네트워크상 참여자들 간에도 믿을 수 있는 거래가 가능하다.', '현재 금융거래만 가능하지만, 곧 모든 거래를 할 수 있을 것이다.']


### AIHUB 대규모 구매도서 기반 한국어 말뭉치 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624)

In [508]:
path_list = ['AIHUB_대규모 구매도서 기반 한국어 말뭉치 데이터/sample/라벨링데이터/000/'+ '/**/*.json']
file_name = json_file_name_list(path_list)
for i in file_name:
    if 'INFO' in i:
        file_name.remove(i)

In [586]:
source_file = file_name[5]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
source_df = pd.DataFrame(one_json_sample['paragraphs'])
source_dict = dict(source_df['sentences'].explode())
source_json = pd.json_normalize(source_dict)  

source_list = []
for source_index in source_json:
    for source_nested_list in source_json[source_index]:

        try:
            for source_single_list in source_nested_list:
                try:
                    for key, value in source_single_list.items():
                        if key == 'text':
                            source_list.append(value)
                except:
                    pass
        except:
            pass  

AIHUB_대규모 구매도서 기반 한국어 말뭉치 데이터/sample/라벨링데이터/000\BOOK_CORPUS_000_TEXT_0006.json


In [ ]:
num = 900

for i in range(num-50, num):
    try : 
        preprocessing_sentences = formal_preprocessing_text_kss3(source_list[i])
        if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
            print("[Before]")
            print(source_list[i])
            print("\n")

            print("[After]")
            for preprocessing_sentence in preprocessing_sentences:          
                    print(preprocessing_sentence, end="\n")
                    print("\n")
    except:
        pass

In [588]:
num = -1
for source in source_list:
    num += 1
    if "물 한 모금" in source:
        print(num)

9891


In [589]:
source_list[9891]

'물 한 모금 입에 넣지 않고 인사불성으로 앓는 중에 “우네.” “아이구, 또 우네.” “젖 얻어먹이러 가야겠다.” 오, 정말 눈물 없인 보기 어려운 장면이다.'

In [565]:
print(source_list[9891])

비는 “아직은 젊고 어리석으므로 내 삶이 후회 없는 삶이 될 수 있을까?라는 의문이 든다. 정답은 없는 것 같지만 비관적인 생각을 버리고 재활운동 열심히 해야겠다. 지금의 시련과 고통을 극복하고 나를 믿고 자신감을 가져 ‘장애’라는 새로운 환경에 잘 적응하여 행복하게 살고 싶다”고 하였다.


In [590]:
preprocessing_sentences = formal_preprocessing_text_kss3(source_list[9891])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

물 한 모금 입에 넣지 않고 인사불성으로 앓는 중에 우네.

아이구, 또 우네.

젖 얻어먹이러 가야겠다.

오, 정말 눈물 없인 보기 어려운 장면이다.



In [591]:
print(preprocessing_sentences)

['물 한 모금 입에 넣지 않고 인사불성으로 앓는 중에 우네.', '아이구, 또 우네.', '젖 얻어먹이러 가야겠다.', '오, 정말 눈물 없인 보기 어려운 장면이다.']


### AIHUB 도서자료 기계독해

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=92)

In [297]:
path_list = ['AIHUB_도서자료 기계독해/'+ '/**/*.json']
file_name = json_file_name_list(path_list)

In [593]:
source_file = file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

source_filter_02 = source_json.filter(regex='[0123456789]').values[0]
source_df_02 = pd.DataFrame(source_filter_02)

for source_df_02_value in source_df_02.values:
    for value in source_df_02_value[0]:
        try:
            source_list.append(value['context'])
        except:
            pass

AIHUB_도서자료 기계독해\Validation\도서.json


In [ ]:
num = 3110

for i in range(num-50, num):
    try : 
        preprocessing_sentences = formal_preprocessing_text_kss3(source_list[i])
        if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
            print("[Before]")
            print(source_list[i])
            print("\n")

            print("[After]")
            for preprocessing_sentence in preprocessing_sentences:          
                    print(preprocessing_sentence, end="\n")
                    print("\n")
    except:
        pass

In [601]:
num = -1
for source in source_list:
    num += 1
    if "① 데이터 규모(Volume)" in source:
        print(num)

3070


In [608]:
source_list[3070]

'개념정의에서도 잠시 언급된 바와 같이 빅 데이터는 ① 데이터 규모(Volume), ② 데이터 처리 및 분석 속도(Velocity), ③ 데이터의 다양성(Variety)의 측면에서 기존 데이터와 차별성을 가지며, 최근에는 빅 데이터의 활용 증대를 반영하여 ④ 데이터의 가치(Value)까지 그 특성으로 언급되고 있다. 그리고 한편으로는 빅 데이터의 ‘고해상성’, ‘비정형성’, ‘실시간성’에 주목하여 기존 데이터와 구별하기도 한다. 특히, 정보는 그 정형화의 정도에 따라 ⓐ 고정된 필드(예컨대, 관계형 데이터베이스, 스프레드시트 등)에 저장된 ‘정형(Structured) 데이터’, ⓑ 고정되지는 않았지만 메타데이터나 스키마 등을 포함(예컨대, XML, HTML 등)하는 ‘반정형(Semi-structured) 데이터’, ⓒ 고정된 필드에 저장되어 있지 않은 ‘비정형 (Unstructured) 데이터(예컨대, 일반 문서, 이미지, 음성, 동영상 등)’ 등으로 세분화 되는데, 빅 데이터는 정형, 반정형, 비정형 데이터 모두를 포함하고, 특히 반정형 및 비정형 데이터를 광범위하게 수집･분석한다는 점에서 ‘다양성’ 내지 ‘비정형성’ 을 갖는다.'

In [602]:
print(source_list[3070])

개념정의에서도 잠시 언급된 바와 같이 빅 데이터는 ① 데이터 규모(Volume), ② 데이터 처리 및 분석 속도(Velocity), ③ 데이터의 다양성(Variety)의 측면에서 기존 데이터와 차별성을 가지며, 최근에는 빅 데이터의 활용 증대를 반영하여 ④ 데이터의 가치(Value)까지 그 특성으로 언급되고 있다. 그리고 한편으로는 빅 데이터의 ‘고해상성’, ‘비정형성’, ‘실시간성’에 주목하여 기존 데이터와 구별하기도 한다. 특히, 정보는 그 정형화의 정도에 따라 ⓐ 고정된 필드(예컨대, 관계형 데이터베이스, 스프레드시트 등)에 저장된 ‘정형(Structured) 데이터’, ⓑ 고정되지는 않았지만 메타데이터나 스키마 등을 포함(예컨대, XML, HTML 등)하는 ‘반정형(Semi-structured) 데이터’, ⓒ 고정된 필드에 저장되어 있지 않은 ‘비정형 (Unstructured) 데이터(예컨대, 일반 문서, 이미지, 음성, 동영상 등)’ 등으로 세분화 되는데, 빅 데이터는 정형, 반정형, 비정형 데이터 모두를 포함하고, 특히 반정형 및 비정형 데이터를 광범위하게 수집･분석한다는 점에서 ‘다양성’ 내지 ‘비정형성’ 을 갖는다.


In [605]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[3070])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

개념정의에서도 잠시 언급된 바와 같이 빅 데이터는   데이터 규모(Volume),   데이터 처리 및 분석 속도(Velocity),   데이터의 다양성(Variety)의 측면에서 기존 데이터와 차별성을 가지며, 최근에는 빅 데이터의 활용 증대를 반영하여   데이터의 가치(Value)까지 그 특성으로 언급되고 있다.

그리고 한편으로는 빅 데이터의  고해상성 ,  비정형성 ,  실시간성 에 주목하여 기존 데이터와 구별하기도 한다.

특히, 정보는 그 정형화의 정도에 따라   고정된 필드(예컨대, 관계형 데이터베이스, 스프레드시트 등)에 저장된  정형(Structured) 데이터 ,   고정되지는 않았지만 메타데이터나 스키마 등을 포함(예컨대, XML, HTML 등)하는  반정형(Semi-structured) 데이터 ,   고정된 필드에 저장되어 있지 않은  비정형 (Unstructured) 데이터(예컨대, 일반 문서, 이미지, 음성, 동영상 등)  등으로 세분화 되는데, 빅 데이터는 정형, 반정형, 비정형 데이터 모두를 포함하고, 특히 반정형 및 비정형 데이터를 광범위하게 수집 분석한다는 점에서  다양성  내지  비정형성  을 갖는다.



In [606]:
print(preprocessing_sentences)

['개념정의에서도 잠시 언급된 바와 같이 빅 데이터는   데이터 규모(Volume),   데이터 처리 및 분석 속도(Velocity),   데이터의 다양성(Variety)의 측면에서 기존 데이터와 차별성을 가지며, 최근에는 빅 데이터의 활용 증대를 반영하여   데이터의 가치(Value)까지 그 특성으로 언급되고 있다.', '그리고 한편으로는 빅 데이터의  고해상성 ,  비정형성 ,  실시간성 에 주목하여 기존 데이터와 구별하기도 한다.', '특히, 정보는 그 정형화의 정도에 따라   고정된 필드(예컨대, 관계형 데이터베이스, 스프레드시트 등)에 저장된  정형(Structured) 데이터 ,   고정되지는 않았지만 메타데이터나 스키마 등을 포함(예컨대, XML, HTML 등)하는  반정형(Semi-structured) 데이터 ,   고정된 필드에 저장되어 있지 않은  비정형 (Unstructured) 데이터(예컨대, 일반 문서, 이미지, 음성, 동영상 등)  등으로 세분화 되는데, 빅 데이터는 정형, 반정형, 비정형 데이터 모두를 포함하고, 특히 반정형 및 비정형 데이터를 광범위하게 수집 분석한다는 점에서  다양성  내지  비정형성  을 갖는다.']


### AIHUB 법률 규정 (판결서 약관 등) 텍스트 분석 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=116&topMenu=100&aihubDataSe=ty&dataSetSn=580)

In [34]:
path_list = ['AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training/' + '/**/*.json',
             'AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation/' + '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [35]:
source_file = train_file_name[6511]
print(source_file)

with open(source_file , 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

    source_df = pd.DataFrame(one_json_sample)
    source_list = source_df.loc['disposalcontent']['disposal'] + \
        source_df.loc['rqestObjet']['mentionedItems'] + \
        source_df.loc['acusrAssrs']['assrs'] + \
        source_df.loc['dedatAssrs']['assrs'] + \
        source_df.loc['bsisFacts']['facts'] + \
        source_df.loc['courtDcss']['dcss'] + \
        source_df.loc['cnclsns']['close']      

AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링데이터\1.Training\라벨링데이터\TL_1.판결문\01.민사\2018\2017브35.json


In [36]:
num = -1
for train_file in train_file_name:
    num += 1
    if "102_공급계약_가공.json" in train_file:
        print(num)

4174


In [37]:
num = 20

for i in range(0, num):
    try : 
        preprocessing_sentences = legislation_preprocessing_text(source_list[i])
        if type(preprocessing_sentences) == list and preprocessing_sentences[0] != source_list[i]:
            print("[Before]")
            print(source_list[i])
            print("\n")

            print("[After]")
            for preprocessing_sentence in preprocessing_sentences:          
                    print(preprocessing_sentence, end="\n")
                    print("\n")
    except:
        pass

[Before]
신청인은 신청외 1과 법률혼 관계에 있던 중 중화인민공화국인인 신청외 2와 교제하게 되었고, 2007. 11. 27. 신청인과 신청외 2 사이에서 사건본인이 태어났다.


[After]
신청인과 신청외 2 사이에서 사건본인이 태어났다.


[Before]
신청인과 신청외 2는 신청외 2의 지인인 신청외 4에게 부탁하여 2007. 12. 6. 사건본인이 신청외 3과 신청외 4 사이에 태어난 딸인 것처럼 출생신고를 하였다.


[After]
사건본인이 신청외 3과 신청외 4 사이에 태어난 딸인 것처럼 출생신고를 하였다.


[Before]
가족관계의 등록 등에 관한 법률(이하 '법'이라 한다) 제104조 는 가정법원의 허가에 의한 가족관계등록부의 정정신청에 관하여, 제107조 는 확정판결에 의한 가족관계등록부의 정정신청에 관하여 각각 규정하고 있다. 


[After]
가족관계의 등록 등에 관한 법률(이하  법 이라 한다) 제104조 는 가정법원의 허가에 의한 가족관계등록부의 정정신청에 관하여, 제107조 는 확정판결에 의한 가족관계등록부의 정정신청에 관하여 각각 규정하고 있다.


[Before]
법원의 허가에 의한 가족관계등록부의 정정은 그 절차의 간이성에 비추어 정정할 사항이 경미한 경우에 허용되는 것이므로, 친족법상 또는 상속법상 중대한 영향을 미칠 수 있는 사항에 대하여는 법 제107조 에 따라 확정판결에 의하여만 가족관계등록부의 정정신청을 할 수 있다( 대법원 1993. 5. 22.자 93스14, 15, 16 결정 등 참조).


[After]
법원의 허가에 의한 가족관계등록부의 정정은 그 절차의 간이성에 비추어 정정할 사항이 경미한 경우에 허용되는 것이므로, 친족법상 또는 상속법상 중대한 영향을 미칠 수 있는 사항에 대하여는 법 제107조 에 따라 확정판결에 의하여만 가족관계등록부의 정정신청을 할 수 있다.


[Before]
이와 같이 국적법은 출생에 의한 국적 취득과 인지에 의한 국적 취득을 별도로 규율하고 있다. 또한 혼인 외의 

In [52]:
source_list[15]

'이와 같이 국적법은 출생에 의한 국적 취득과 인지에 의한 국적 취득을 별도로 규율하고 있다. 또한 혼인 외의 자의 경우 모와의 관계에서는 출산과 동시에 친자관계가 발생하지만, 친부(친부)와의 관계에서는 인지 없이는 법률상 친자관계가 발생하지 않는다( 대법원 1966. 11. 29. 선고 66다1251 판결 , 대법원 1984. 9. 25. 선고 84므73 판결 등 참조). '

In [53]:
print(source_list[15])

이와 같이 국적법은 출생에 의한 국적 취득과 인지에 의한 국적 취득을 별도로 규율하고 있다. 또한 혼인 외의 자의 경우 모와의 관계에서는 출산과 동시에 친자관계가 발생하지만, 친부(친부)와의 관계에서는 인지 없이는 법률상 친자관계가 발생하지 않는다( 대법원 1966. 11. 29. 선고 66다1251 판결 , 대법원 1984. 9. 25. 선고 84므73 판결 등 참조). 


In [54]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[15])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

이와 같이 국적법은 출생에 의한 국적 취득과 인지에 의한 국적 취득을 별도로 규율하고 있다.

또한 혼인 외의 자의 경우 모와의 관계에서는 출산과 동시에 친자관계가 발생하지만, 친부(친부)와의 관계에서는 인지 없이는 법률상 친자관계가 발생하지 않는다.



In [55]:
print(preprocessing_sentences)

['이와 같이 국적법은 출생에 의한 국적 취득과 인지에 의한 국적 취득을 별도로 규율하고 있다.', '또한 혼인 외의 자의 경우 모와의 관계에서는 출산과 동시에 친자관계가 발생하지만, 친부(친부)와의 관계에서는 인지 없이는 법률상 친자관계가 발생하지 않는다.']


In [ ]:
source_file = train_file_name[5014]
print(source_file)

with open(source_file , 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

    source_df = pd.DataFrame(one_json_sample)
    source_list = source_df.loc['disposalcontent']['disposal'] + \
        source_df.loc['rqestObjet']['mentionedItems'] + \
        source_df.loc['acusrAssrs']['assrs'] + \
        source_df.loc['dedatAssrs']['assrs'] + \
        source_df.loc['bsisFacts']['facts'] + \
        source_df.loc['courtDcss']['dcss'] + \
        source_df.loc['cnclsns']['close']      

In [ ]:
print(source_list[5014])

In [551]:
preprocessing_sentences = formal_preprocessing_text(source_list[9])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

청구인은, 0000 (주소 1 생략) ㅇ동 ㅁㅁㅁ호, ㅁㅁㅁ호, ㅁㅁㅁ호, 0000 (주소 2 생략) ㅁ동 ㅁㅁㅁ호 전세보증금 9,000만 원 반환채권, 0000 (주소 3 생략) ㅇㅇㅇ호는 소외 1이 소외 2, 소외 3, 소외 4, 소외 5에게 명의신탁한 재산이므로 이 부분도 재산분할대상에 포함되어야 한다고 주장한다.



In [620]:
source_file = train_file_name[5014]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame.from_dict(one_json_sample, orient='index')
source_df = source_df.transpose()
source_df = pd.DataFrame(one_json_sample)
source_list = [source_df['clauseArticle'][0], source_df['comProvision'][0]]

AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링데이터\1.Training\라벨링데이터\TL_2.약관\01.유리\037_산후조리원_가공.json


In [624]:
source_list[0]

'제6조 (이용계약의 변경 및 조정)\n① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. 다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.'

In [621]:
print(source_list[0])

제6조 (이용계약의 변경 및 조정)
① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. 다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.


In [622]:
preprocessing_sentences = legislation_preprocessing_text(source_list[0])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다.

다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.



In [623]:
print(preprocessing_sentences)

['이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다.', '다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.']


In [626]:
source_file = train_file_name[4174]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame.from_dict(one_json_sample, orient='index')
source_df = source_df.transpose()
source_list = [source_df['clauseArticle'][0][0], source_df['illdcssBasiss'][0][0],
                source_df['relateLaword'][0][0]]   

AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링데이터\1.Training\라벨링데이터\TL_2.약관\02.불리\102_공급계약_가공.json


In [638]:
source_list[2]

'제8조(손해배상액의 예정) \n고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.'

In [639]:
print(source_list[2])

제8조(손해배상액의 예정) 
고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.


In [640]:
preprocessing_sentences = legislation_preprocessing_text(source_list[2])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.



In [641]:
print(preprocessing_sentences)

['고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.']


### AIHUB 일반상식

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=116&topMenu=100&aihubDataSe=ty&dataSetSn=106)

In [23]:
path_list = ['AIHUB_일반상식/'+ '/**/*.json']
file_name_list = json_file_name_list(path_list)

In [24]:
for file_name in file_name_list:
    if 'ko_wiki_v1_squad' in file_name:
        source_file = file_name

In [25]:
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_일반상식\ko_wiki_v1_squad.json


In [33]:
one_json_sample['data'][9961]

{'paragraphs': [{'qas': [{'question': '처음 핑크베리 열었을 때가 몇 년도야',
     'answers': [{'answer_start': 114, 'text': '2005년'}],
     'id': '9_f2_wiki_2863-1'}],
   'context': '2014년 11월 현재, 점포 수는 19개 나라, 미국 27개 주에 걸쳐 250개이다. 핑크베리의 점포는 대부분 캘리포니아 주 남부에 있다. 뉴욕 시에도 16개의 점포가 있다. 핑크베리의 첫 번째 점포는 2005년 1월 문을 열었다. 한국계 미국인 황혜경 이 창업하였다. 일부 사람들은 핑크베리의 제품들을 가리켜 "크랙베리"(Crackberry), "얼린 헤로인 주스"(frozen heroin juice)라고 불렀다. (크랙베리라고 별명이 붙은 것에는 또한 유명한 휴대전화 블랙베리가 있다.) 핑크베리사는 그 회사의 고객들이 "컬트" 취향인 것을 잘 인지하고서는, 회사 홈페이지에 "그루피 코너"(groupie corner)라는 것을 두고 있다.'}],
 'title': '핑크베리'}

In [26]:
source_list[9961]

'2014년 11월 현재, 점포 수는 19개 나라, 미국 27개 주에 걸쳐 250개이다. 핑크베리의 점포는 대부분 캘리포니아 주 남부에 있다. 뉴욕 시에도 16개의 점포가 있다. 핑크베리의 첫 번째 점포는 2005년 1월 문을 열었다. 한국계 미국인 황혜경 이 창업하였다. 일부 사람들은 핑크베리의 제품들을 가리켜 "크랙베리"(Crackberry), "얼린 헤로인 주스"(frozen heroin juice)라고 불렀다. (크랙베리라고 별명이 붙은 것에는 또한 유명한 휴대전화 블랙베리가 있다.) 핑크베리사는 그 회사의 고객들이 "컬트" 취향인 것을 잘 인지하고서는, 회사 홈페이지에 "그루피 코너"(groupie corner)라는 것을 두고 있다.'

In [27]:
print(source_list[9961])

2014년 11월 현재, 점포 수는 19개 나라, 미국 27개 주에 걸쳐 250개이다. 핑크베리의 점포는 대부분 캘리포니아 주 남부에 있다. 뉴욕 시에도 16개의 점포가 있다. 핑크베리의 첫 번째 점포는 2005년 1월 문을 열었다. 한국계 미국인 황혜경 이 창업하였다. 일부 사람들은 핑크베리의 제품들을 가리켜 "크랙베리"(Crackberry), "얼린 헤로인 주스"(frozen heroin juice)라고 불렀다. (크랙베리라고 별명이 붙은 것에는 또한 유명한 휴대전화 블랙베리가 있다.) 핑크베리사는 그 회사의 고객들이 "컬트" 취향인 것을 잘 인지하고서는, 회사 홈페이지에 "그루피 코너"(groupie corner)라는 것을 두고 있다.


In [28]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[9961])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

2014년 11월 현재, 점포 수는 19개 나라, 미국 27개 주에 걸쳐 250개이다.

핑크베리의 점포는 대부분 캘리포니아 주 남부에 있다.

핑크베리의 첫 번째 점포는 2005년 1월 문을 열었다.

일부 사람들은 핑크베리의 제품들을 가리켜  크랙베리 (Crackberry),  얼린 헤로인 주스 (frozen heroin juice)라고 불렀다.

크랙베리라고 별명이 붙은 것에는 또한 유명한 휴대전화 블랙베리가 있다.

핑크베리사는 그 회사의 고객들이  컬트  취향인 것을 잘 인지하고서는, 회사 홈페이지에  그루피 코너 (groupie corner)라는 것을 두고 있다.



In [29]:
print(preprocessing_sentences)

['2014년 11월 현재, 점포 수는 19개 나라, 미국 27개 주에 걸쳐 250개이다.', '핑크베리의 점포는 대부분 캘리포니아 주 남부에 있다.', '핑크베리의 첫 번째 점포는 2005년 1월 문을 열었다.', '일부 사람들은 핑크베리의 제품들을 가리켜  크랙베리 (Crackberry),  얼린 헤로인 주스 (frozen heroin juice)라고 불렀다.', '크랙베리라고 별명이 붙은 것에는 또한 유명한 휴대전화 블랙베리가 있다.', '핑크베리사는 그 회사의 고객들이  컬트  취향인 것을 잘 인지하고서는, 회사 홈페이지에  그루피 코너 (groupie corner)라는 것을 두고 있다.']


In [647]:
print(source_list[619])

〈밤 어딘가에서 ~night shift~〉(일본어: 夜のどこかで 〜night shift〜 요루노도코카데 나이트시프트)는 일본의 여가수 나카모리 아키나(中森明菜)의 29번째 싱글로 1994년 9월 2일에 발매하였다. (8cmCD: MVDD-10007) 작사는 나츠노 세리코(夏野芹子)가, 작곡과 편곡은 고토 츠구토시(後藤次利)가 담당하였고 MCA 빅터의 레이블로 출시되었다. B사이드 곡은 〈Rose Bud〉로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다. 이 곡은 뉴스 프로그램인 《NNN 오늘의 사건》(NNNきょうの出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다. 나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다. 1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다.


In [648]:
preprocessing_sentences = formal_preprocessing_text_kss1(source_list[619])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

B사이드 곡은  Rose Bud 로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다.

이 곡은 뉴스 프로그램인  NNN 오늘의 사건 (NNN    出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다.

나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다.

1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다.

